## Applied Data Science Capstone

This notebook that is used for the Applied Data Science Capstone Project with the objective of acquiring new skills.

In [185]:

import pandas as pd
import numpy as np
import re
#!conda install -c conda-forge python-levenshtein # for determining the amount of text operations necessary to
                                                  # transform 1 string into the other
#!conda install -c conda-forge pymc3 -y # for the creation of negative binomial models in python
from Levenshtein import distance as levenshtein_distance



print("Hello Capstone Project Course!")

Hello Capstone Project Course!


### Data Preparation

In [186]:
pwd

'/home/sbr0/Dropbox/Analysis Planning/coursera stuffs/Capstone/First-Class Restaurant Location Analysis'

In [187]:
years = [2015,2016,2017]
crime_data_set_name = '/home/sbr0/Dropbox/Analysis Planning/coursera stuffs/Capstone/First-Class Restaurant Location Analysis/data sets/Offenses_Known_to_Law_Enforcement_by_State_by_Metropolitan_and_Nonmetropolitan_Counties_{year}.xls'
crimes_dict = {year:pd.read_excel(crime_data_set_name.format(year = year)) for year in years}
for year in crimes_dict:
    crimes_dict[year] = crimes_dict[year].rename(columns=crimes_dict[year].loc[3]).drop([0,1,2,3]).reset_index(drop=True)
    crimes_dict[year].State = crimes_dict[year].State.ffill()
    display(crimes_dict[year].head())

,State,County,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape\n(revised\ndefinition)1,Rape\n(legacy\ndefinition)2,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson3
0,ALABAMA - Metropolitan Counties,Autauga,69,0,13,NaN,6,50,344,111,187,46,NaN
1,ALABAMA - Metropolitan Counties,Baldwin,115,0,9,NaN,33,73,648,225,390,33,NaN
2,ALABAMA - Metropolitan Counties,Bibb,7,0,2,NaN,1,4,41,20,18,3,NaN
3,ALABAMA - Metropolitan Counties,Blount,204,5,16,NaN,5,178,832,247,503,82,NaN
4,ALABAMA - Metropolitan Counties,Calhoun,16,0,4,NaN,1,11,413,181,225,7,NaN


,State,County,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape\n(revised\ndefinition)1,Rape\n(legacy\ndefinition)2,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson3
0,ALABAMA - Metropolitan Counties,Autauga,73,1,12,NaN,8,52,429,146,233,50,NaN
1,ALABAMA - Metropolitan Counties,Baldwin,127,1,5,NaN,23,98,613,229,342,42,NaN
2,ALABAMA - Metropolitan Counties,Bibb,0,0,0,NaN,0,0,37,20,14,3,NaN
3,ALABAMA - Metropolitan Counties,Blount,394,1,17,NaN,9,367,867,261,501,105,NaN
4,ALABAMA - Metropolitan Counties,Calhoun,23,0,7,NaN,5,11,319,137,181,1,NaN


,State,County,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape1,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson2
0,ALABAMA - Metropolitan Counties,Autauga,55,2,5,5,43,384,131,206,47,NaN
1,ALABAMA - Metropolitan Counties,Baldwin,163,1,9,18,135,737,226,442,69,NaN
2,ALABAMA - Metropolitan Counties,Bibb,3,0,1,1,1,69,41,24,4,NaN
3,ALABAMA - Metropolitan Counties,Blount,313,1,14,5,293,812,198,502,112,NaN
4,ALABAMA - Metropolitan Counties,Calhoun,18,0,7,7,4,211,93,117,1,NaN


In [188]:
for year, df in crimes_dict.items():
    print(year)
    display(df.head(10))

2015


,State,County,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape\n(revised\ndefinition)1,Rape\n(legacy\ndefinition)2,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson3
0,ALABAMA - Metropolitan Counties,Autauga,69,0,13,NaN,6,50,344,111,187,46,NaN
1,ALABAMA - Metropolitan Counties,Baldwin,115,0,9,NaN,33,73,648,225,390,33,NaN
2,ALABAMA - Metropolitan Counties,Bibb,7,0,2,NaN,1,4,41,20,18,3,NaN
3,ALABAMA - Metropolitan Counties,Blount,204,5,16,NaN,5,178,832,247,503,82,NaN
4,ALABAMA - Metropolitan Counties,Calhoun,16,0,4,NaN,1,11,413,181,225,7,NaN
5,ALABAMA - Metropolitan Counties,Chilton,120,0,23,NaN,1,96,575,263,274,38,NaN
6,ALABAMA - Metropolitan Counties,Elmore,93,5,17,NaN,5,66,1141,495,574,72,NaN
7,ALABAMA - Metropolitan Counties,Etowah,113,1,23,NaN,2,87,561,214,298,49,NaN
8,ALABAMA - Metropolitan Counties,Geneva,41,0,9,NaN,0,32,229,91,113,25,NaN
9,ALABAMA - Metropolitan Counties,Henry,36,0,8,NaN,3,25,162,78,64,20,NaN


2016


,State,County,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape\n(revised\ndefinition)1,Rape\n(legacy\ndefinition)2,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson3
0,ALABAMA - Metropolitan Counties,Autauga,73,1,12,NaN,8,52,429,146,233,50,NaN
1,ALABAMA - Metropolitan Counties,Baldwin,127,1,5,NaN,23,98,613,229,342,42,NaN
2,ALABAMA - Metropolitan Counties,Bibb,0,0,0,NaN,0,0,37,20,14,3,NaN
3,ALABAMA - Metropolitan Counties,Blount,394,1,17,NaN,9,367,867,261,501,105,NaN
4,ALABAMA - Metropolitan Counties,Calhoun,23,0,7,NaN,5,11,319,137,181,1,NaN
5,ALABAMA - Metropolitan Counties,Chilton,151,0,10,NaN,3,138,592,247,295,50,NaN
6,ALABAMA - Metropolitan Counties,Elmore,88,4,22,NaN,7,55,677,223,408,46,NaN
7,ALABAMA - Metropolitan Counties,Etowah,135,1,20,NaN,7,107,480,149,255,76,NaN
8,ALABAMA - Metropolitan Counties,Geneva,46,1,9,NaN,2,34,205,71,106,28,NaN
9,ALABAMA - Metropolitan Counties,Hale,33,0,2,NaN,2,29,139,73,55,11,NaN


2017


,State,County,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape1,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson2
0,ALABAMA - Metropolitan Counties,Autauga,55,2,5,5,43,384,131,206,47,NaN
1,ALABAMA - Metropolitan Counties,Baldwin,163,1,9,18,135,737,226,442,69,NaN
2,ALABAMA - Metropolitan Counties,Bibb,3,0,1,1,1,69,41,24,4,NaN
3,ALABAMA - Metropolitan Counties,Blount,313,1,14,5,293,812,198,502,112,NaN
4,ALABAMA - Metropolitan Counties,Calhoun,18,0,7,7,4,211,93,117,1,NaN
5,ALABAMA - Metropolitan Counties,Chilton,168,2,15,7,144,715,268,368,79,NaN
6,ALABAMA - Metropolitan Counties,Colbert,58,0,13,3,42,267,84,151,32,NaN
7,ALABAMA - Metropolitan Counties,Elmore,60,1,13,9,37,757,228,467,62,NaN
8,ALABAMA - Metropolitan Counties,Etowah,104,2,13,3,86,526,186,276,64,NaN
9,ALABAMA - Metropolitan Counties,Geneva,58,0,7,3,48,227,73,124,30,NaN


In [189]:
crimes = ['Violent\ncrime'
'Murder and\nnonnegligent\nmanslaughter'
'Rape1'
'Rape\n(revised\ndefinition)1'
'Rape\n(legacy\ndefinition)2'
'Robbery'
'Aggravated\nassault'
'Property\ncrime'
'Burglary'
'Arson2'
'Arson3']

In [190]:
for year, df in crimes_dict.items():
    print(year)
    display(df.tail(11))

2015


,State,County,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape\n(revised\ndefinition)1,Rape\n(legacy\ndefinition)2,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson3
2584,WYOMING - Nonmetropolitan Counties,Uinta,3,0,2,NaN,0,1,88,2,78,8,0
2585,WYOMING - Nonmetropolitan Counties,Washakie,0,0,0,NaN,0,0,3,0,2,1,0
2586,WYOMING - Nonmetropolitan Counties,Weston,5,0,0,NaN,0,5,7,2,4,1,0
2587,1 The figures shown in this column for the offense of rape were reported using the revised Uniform Crime Reporting (...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2588,2 The figures shown in this column for the offense of rape were reported using the legacy UCR definition of rape. Se...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2589,3 The FBI does not publish arson data unless it receives data from either the agency or the state for all 12 months ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2590,"4 This agency began the year submitting rape data classified according to the legacy UCR definition. However, at som...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2591,5 The Tulare County Highway Patrol collects the motor vehicle thefts for this county. These data can be found in Tab...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2592,"6 The FBI determined that the agency's data were underreported. Consequently, those data are not included in this ta...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2593,"7 Because of changes in the state/local agency's reporting practices, figures are not comparable to previous years' ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


2016


,State,County,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape\n(revised\ndefinition)1,Rape\n(legacy\ndefinition)2,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson3
2610,WYOMING - Nonmetropolitan Counties,Washakie,0,0,0,NaN,0,0,14,4,7,3,0
2611,WYOMING - Nonmetropolitan Counties,Weston,1,0,0,NaN,0,1,3,0,3,0,0
2612,1 The figures shown in this column for the offense of rape were reported using the revised Uniform Crime Reporting (...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2613,2 The figures shown in this column for the offense of rape were reported using the legacy UCR definition of rape. Se...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2614,3 The FBI does not publish arson data unless it receives data from either the agency or the state for all 12 months ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2615,4 The Tulare County Highway Patrol collects the motor vehicle thefts for this county. These data can be found in Tab...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2616,"5 The FBI determined that the agency's data were underreported. Consequently, those data are not included in this ta...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2617,6 The FBI determined that the agency did not follow national UCR Program guidelines for reporting an offense. Conse...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2618,"7 This agency began the year submitting rape data classified according to the legacy UCR definition. However, at som...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2619,"8 Because of changes in the state/local agency's reporting practices, figures are not comparable to previous years' ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


2017


,State,County,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape1,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson2
2513,WYOMING - Nonmetropolitan Counties,Sweetwater,23,1,3,0,19,99,24,63,12,1
2514,WYOMING - Nonmetropolitan Counties,Uinta,2,0,0,0,2,78,7,64,7,0
2515,WYOMING - Nonmetropolitan Counties,Washakie,4,0,2,0,2,11,3,8,0,0
2516,WYOMING - Nonmetropolitan Counties,Weston,2,0,0,0,2,2,0,2,0,0
2517,1 The figures shown in this column for the offense of rape were reported using only the revised Uniform Crime Report...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2518,2 The FBI does not publish arson data unless it receives data from either the agency or the state for all 12 months ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2519,"3 The FBI determined that the agency's data were underreported. Consequently, those data are not included in this ta...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2520,4 The Tulare County Highway Patrol collects the motor vehicle thefts for this county. These data can be found in Tab...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2521,"5 The FBI determined that the agency's data were overreported. Consequently, those data are not included in this table.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2522,"6 Because of changes in the state/local agency's reporting practices, figures are not comparable to previous years' ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [191]:
notes_txt = "1 The figures shown in this column for..."
notes_idx_dict = {year:crime_df.loc[crime_df.iloc[:,0].str.contains(notes_txt).fillna(False)].index for year, crime_df in crimes_dict.items()}
notices_dict = {}
for year in notes_idx_dict:
    assert notes_idx_dict[year].size == 1
    notes_idx_dict[year]=notes_idx_dict[year].values[0]
    notices_dict[year]=crimes_dict[year].iloc[notes_idx_dict[year]:,0].values.tolist()
    print('{}:'.format(year))
    print(*notices_dict[year],sep='\n')
    print()


2015:
1 The figures shown in this column for the offense of rape were reported using the revised Uniform Crime Reporting (UCR) definition of rape. See the data declaration for further explanation.
2 The figures shown in this column for the offense of rape were reported using the legacy UCR definition of rape. See the data declaration for further explanation.
3 The FBI does not publish arson data unless it receives data from either the agency or the state for all 12 months of the calendar year.
4 This agency began the year submitting rape data classified according to the legacy UCR definition. However, at some point during the calendar year, the agency modified its reporting methods and began classifying and submitting rape offenses according to the revised UCR definition of rape. See the data declaration for further explanation.
5 The Tulare County Highway Patrol collects the motor vehicle thefts for this county. These data can be found in Table 11. 
6 The FBI determined that the agenc

In [192]:
notices_dict

{2015: ['1 The figures shown in this column for the offense of rape were reported using the revised Uniform Crime Reporting (UCR) definition of rape. See the data declaration for further explanation.',
  '2 The figures shown in this column for the offense of rape were reported using the legacy UCR definition of rape. See the data declaration for further explanation.',
  '3 The FBI does not publish arson data unless it receives data from either the agency or the state for all 12 months of the calendar year.',
  '4 This agency began the year submitting rape data classified according to the legacy UCR definition. However, at some point during the calendar year, the agency modified its reporting methods and began classifying and submitting rape offenses according to the revised UCR definition of rape. See the data declaration for further explanation.',
  '5 The Tulare County Highway Patrol collects the motor vehicle thefts for this county. These data can be found in Table 11. ',
  "6 The F

In [193]:
notice_summaries = {}

notice_summaries[2015]={
'1': "rape def (revised)",
'2':"rape def (previous)",
'3':"arson data publication restricted to reports all 12 months",
'4':"rape classification switched to revised def during year",
'5':"motor vehicle thefts reported by Tulare County Highway Patrol",
'6': "Data not recorded due to under reporting",
'7':"Reporting Practices not comparable to previous year's data",
'8':"Data not recorded due to over reporting"}


notice_summaries[2016]={
    '1': "rape def (revised)",
 '2':"rape def (previous)",
 '3':"arson data publication restricted to reports all 12 months",
'4':"motor vehicle thefts reported by Tulare County Highway Patrol",
'5': "Data not recorded due to under reporting",
'6':"UCR guidelines not followed",
'7':"rape classification switched to revised def during year",
'8':"Reporting Practices not comparable to previous year's data",
'9':"Data not recorded due to over reporting"}

notice_summaries[2017]={
    '1': "rape def (revised)",
    '2':"arson data publication restricted to reports all 12 months",
    '3': "Data not recorded due to under reporting",
    '4':"motor vehicle thefts reported by Tulare County Highway Patrol",
    '5':"Data not recorded due to over reporting",
    '6':"Reporting Practices not comparable to previous year's data",
    '7':"rape def (previous)"
}

def create_table(notice_data,year):
    notice_list = [notice.split(' ',1)+[year] for notice in notice_data]
    notice_df = pd.DataFrame(notice_list,columns=['number','notice','year'])
    return notice_df

notices_table = pd.concat([create_table(notice,year) for year,notice in notices_dict.items()])


for year in crimes_dict:
    notices_table.loc[notices_table['year'] == year,'summary'] = notices_table.loc[notices_table['year'] == year,'number'].map(notice_summaries[year])
pd.options.display.max_colwidth = 120
notices_table
#notices_summary[2015] = {1: "rape def (revised)",
#                        2: ,
#                        3: }
#notices_table.loc[notices_table['year'] == 2016,'number'].astype(int).apply(lambda x: summ_16[x])

,number,notice,year,summary
0,1,The figures shown in this column for the offense of rape were reported using the revised Uniform Crime Reporting (UC...,2015,rape def (revised)
1,2,The figures shown in this column for the offense of rape were reported using the legacy UCR definition of rape. See ...,2015,rape def (previous)
2,3,The FBI does not publish arson data unless it receives data from either the agency or the state for all 12 months of...,2015,arson data publication restricted to reports all 12 months
3,4,"This agency began the year submitting rape data classified according to the legacy UCR definition. However, at some ...",2015,rape classification switched to revised def during year
4,5,The Tulare County Highway Patrol collects the motor vehicle thefts for this county. These data can be found in Table...,2015,motor vehicle thefts reported by Tulare County Highway Patrol
5,6,"The FBI determined that the agency's data were underreported. Consequently, those data are not included in this table.",2015,Data not recorded due to under reporting
6,7,"Because of changes in the state/local agency's reporting practices, figures are not comparable to previous years' da...",2015,Reporting Practices not comparable to previous year's data
7,8,"The FBI determined that the agency's data were overreported. Consequently, those data are not included in this table.",2015,Data not recorded due to over reporting
0,1,The figures shown in this column for the offense of rape were reported using the revised Uniform Crime Reporting (UC...,2016,rape def (revised)
1,2,The figures shown in this column for the offense of rape were reported using the legacy UCR definition of rape. See ...,2016,rape def (previous)


In [194]:
for year in crimes_dict:
    notes_idx = notes_idx_dict[year]
    notes_indices=crimes_dict[year].iloc[notes_idx:].index
    crimes_dict[year].drop(notes_indices,inplace=True)
    display(crimes_dict[year].tail())

,State,County,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape\n(revised\ndefinition)1,Rape\n(legacy\ndefinition)2,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson3
2582,WYOMING - Nonmetropolitan Counties,Sublette,9,0,0,NaN,0,9,111,16,91,4,0
2583,WYOMING - Nonmetropolitan Counties,Sweetwater,34,0,14,NaN,1,19,103,27,62,14,3
2584,WYOMING - Nonmetropolitan Counties,Uinta,3,0,2,NaN,0,1,88,2,78,8,0
2585,WYOMING - Nonmetropolitan Counties,Washakie,0,0,0,NaN,0,0,3,0,2,1,0
2586,WYOMING - Nonmetropolitan Counties,Weston,5,0,0,NaN,0,5,7,2,4,1,0


,State,County,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape\n(revised\ndefinition)1,Rape\n(legacy\ndefinition)2,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson3
2607,WYOMING - Nonmetropolitan Counties,Sublette,3,0,0,NaN,1,2,79,5,69,5,0
2608,WYOMING - Nonmetropolitan Counties,Sweetwater,25,0,7,NaN,0,18,124,33,69,22,5
2609,WYOMING - Nonmetropolitan Counties,Uinta,4,0,0,NaN,0,4,60,5,50,5,0
2610,WYOMING - Nonmetropolitan Counties,Washakie,0,0,0,NaN,0,0,14,4,7,3,0
2611,WYOMING - Nonmetropolitan Counties,Weston,1,0,0,NaN,0,1,3,0,3,0,0


,State,County,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape1,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson2
2512,WYOMING - Nonmetropolitan Counties,Sublette,10,0,0,0,10,81,9,68,4,2
2513,WYOMING - Nonmetropolitan Counties,Sweetwater,23,1,3,0,19,99,24,63,12,1
2514,WYOMING - Nonmetropolitan Counties,Uinta,2,0,0,0,2,78,7,64,7,0
2515,WYOMING - Nonmetropolitan Counties,Washakie,4,0,2,0,2,11,3,8,0,0
2516,WYOMING - Nonmetropolitan Counties,Weston,2,0,0,0,2,2,0,2,0,0


In [195]:
for year in crimes_dict:
    print('                    ',year)
    print('----------------------------------------------')    
    print(crimes_dict[year].isnull().sum())
    print("\n\n")

                     2015
----------------------------------------------
State                                        0
County                                       0
Violent\ncrime                               0
Murder and\nnonnegligent\nmanslaughter       0
Rape\n(revised\ndefinition)1               404
Rape\n(legacy\ndefinition)2               2180
Robbery                                      0
Aggravated\nassault                          0
Property\ncrime                              3
Burglary                                     1
Larceny-\ntheft                              1
Motor\nvehicle\ntheft                        1
Arson3                                     143
dtype: int64



                     2016
----------------------------------------------
State                                        0
County                                       0
Violent\ncrime                               2
Murder and\nnonnegligent\nmanslaughter       0
Rape\n(revised\ndefinition)1           

In [196]:
#crimes_df.iloc[:,2:].astype(float).corr()

In [197]:
crime_renaming_dict={
 'Violent\ncrime':'Violent crime',
 'Murder and\nnonnegligent\nmanslaughter':'Murder and nonnegligent manslaughter',
 'Rape1':'Rape',
 'Rape\n(revised\ndefinition)1':'Rape (revised)',
 'Rape\n(legacy\ndefinition)2':'Rape (previous def)',
 'Robbery':'Robbery',
 'Aggravated\nassault':'Aggravated assault',
 'Property\ncrime':'Property crime',
 'Burglary':'Burglary',
 'Larceny-\ntheft':'Larceny-theft',
 'Motor\nvehicle\ntheft':'Motor vehicle theft',
 'Arson2':'Arson',
 'Arson3':'Arson'}

for year in crimes_dict:
    crimes_dict[year] = crimes_dict[year].rename(columns=crime_renaming_dict)
    display(crimes_dict[year].head())

,State,County,Violent crime,Murder and nonnegligent manslaughter,Rape (revised),Rape (previous def),Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson
0,ALABAMA - Metropolitan Counties,Autauga,69,0,13,NaN,6,50,344,111,187,46,NaN
1,ALABAMA - Metropolitan Counties,Baldwin,115,0,9,NaN,33,73,648,225,390,33,NaN
2,ALABAMA - Metropolitan Counties,Bibb,7,0,2,NaN,1,4,41,20,18,3,NaN
3,ALABAMA - Metropolitan Counties,Blount,204,5,16,NaN,5,178,832,247,503,82,NaN
4,ALABAMA - Metropolitan Counties,Calhoun,16,0,4,NaN,1,11,413,181,225,7,NaN


,State,County,Violent crime,Murder and nonnegligent manslaughter,Rape (revised),Rape (previous def),Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson
0,ALABAMA - Metropolitan Counties,Autauga,73,1,12,NaN,8,52,429,146,233,50,NaN
1,ALABAMA - Metropolitan Counties,Baldwin,127,1,5,NaN,23,98,613,229,342,42,NaN
2,ALABAMA - Metropolitan Counties,Bibb,0,0,0,NaN,0,0,37,20,14,3,NaN
3,ALABAMA - Metropolitan Counties,Blount,394,1,17,NaN,9,367,867,261,501,105,NaN
4,ALABAMA - Metropolitan Counties,Calhoun,23,0,7,NaN,5,11,319,137,181,1,NaN


,State,County,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson
0,ALABAMA - Metropolitan Counties,Autauga,55,2,5,5,43,384,131,206,47,NaN
1,ALABAMA - Metropolitan Counties,Baldwin,163,1,9,18,135,737,226,442,69,NaN
2,ALABAMA - Metropolitan Counties,Bibb,3,0,1,1,1,69,41,24,4,NaN
3,ALABAMA - Metropolitan Counties,Blount,313,1,14,5,293,812,198,502,112,NaN
4,ALABAMA - Metropolitan Counties,Calhoun,18,0,7,7,4,211,93,117,1,NaN


In [198]:
r1='Rape (revised)'
r2='Rape (previous def)'
cr15=crimes_dict[2015]
rp=(cr15[r1].fillna("").astype(str) + cr15[r2].fillna("").astype(str))
print("Total Missing Rape Data (15): ",rp.loc[rp == ''].shape[0])
cr16=crimes_dict[2016]
rp16=(cr16[r1].fillna("").astype(str) + cr16[r2].fillna("").astype(str))
print("Total Missing Rape Data (16): ",rp16.loc[rp16 == ''].shape[0])

Total Missing Rape Data (15):  0
Total Missing Rape Data (16):  1


In [199]:
display(cr15[[r1,r2]].loc[cr15[[r1,r2]].notnull().all(axis=1)])
display(cr16[[r1,r2]].loc[cr16[[r1,r2]].notnull().all(axis=1)])

,Rape (revised),Rape (previous def)
1673,0,45
1676,0,24
2545,0,0


,Rape (revised),Rape (previous def)


In [200]:
# For determining the suitability of parsing metropolitan/nonmetropolitan county status:
for year in crimes_dict:
    crimes_dict[year]['State/County Type'] = crimes_dict[year].State
    print(year," Metropolitan Status parsable:",crimes_dict[year]['State/County Type'].str.contains('- Metropolitan|- Nonmetropolitan').sum() == crimes_dict[year].shape[0] and
     crimes_dict[year]['State/County Type'].apply(lambda item: item.count("-") != 1).sum() == 0)

2015  Metropolitan Status parsable: True
2016  Metropolitan Status parsable: True
2017  Metropolitan Status parsable: True


In [201]:
yearly_states = {}
for year in crimes_dict:
    crimes_dict[year].State = crimes_dict[year]['State/County Type'].str.replace("- [MN].+","").str.title().str.strip()
# note: unique() sorts all items in a list by default
yearly_states = {year:crimes_dict[year].State.unique() for year in crimes_dict}

In [202]:
crime_cols = [col for col in crime_renaming_dict.values() if '(' not in col]
crime_cols

['Violent crime',
 'Murder and nonnegligent manslaughter',
 'Rape',
 'Robbery',
 'Aggravated assault',
 'Property crime',
 'Burglary',
 'Larceny-theft',
 'Motor vehicle theft',
 'Arson',
 'Arson']

In [203]:
from itertools import combinations
# (2015 w 2016, 2016 w 2017, 2015 w 2017)
assert np.all([(yearly_states[y1] == yearly_states[y2]).all() for y1,y2 in combinations([2015,2016,2017],2)]), "Not all states are represented across data tables"
    

In [204]:
#crimes_df.loc[crime_ct.str.contains('Police Department',regex=False)].sort_values(by="County")

for year in crimes_dict:
    crimes_dict[year]=crimes_dict[year].rename(columns={'County':'Raw County Data'})

#match every set of numbers that appear after numbers and are preceded by letters or spaces
    print(year)
    display(crimes_dict[year].loc[crimes_dict[year]['Raw County Data'].str.contains("(?<=[a-z]| )\d+")].head())
    print()

2015


,State,Raw County Data,Violent crime,Murder and nonnegligent manslaughter,Rape (revised),Rape (previous def),Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson,State/County Type
136,California,Butte4,155,4,20,NaN,14,117,1422,708,693,21,0,CALIFORNIA - Metropolitan Counties
138,California,El Dorado4,252,9,46,NaN,32,165,2031,577,1412,42,4,CALIFORNIA - Metropolitan Counties
140,California,Imperial4,219,0,6,NaN,7,206,625,202,395,28,21,CALIFORNIA - Metropolitan Counties
142,California,Kings4,122,2,16,NaN,9,95,470,193,265,12,5,CALIFORNIA - Metropolitan Counties
157,California,San Luis Obispo4,324,3,17,NaN,15,289,1347,461,873,13,3,CALIFORNIA - Metropolitan Counties



2016


,State,Raw County Data,Violent crime,Murder and nonnegligent manslaughter,Rape (revised),Rape (previous def),Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson,State/County Type
165,California,Tulare4,350,19,28,NaN,62,241,2074,756,1318,0,1,CALIFORNIA - Metropolitan Counties
215,Colorado,Eagle5,33,0,14,NaN,0,19,NaN,37,NaN,15,3,COLORADO - Nonmetropolitan Counties
736,Kansas,Cherokee6,32,0,3,NaN,4,25,NaN,NaN,102,21,4,KANSAS - Nonmetropolitan Counties
747,Kansas,Ford6,14,0,1,NaN,0,13,NaN,NaN,41,14,0,KANSAS - Nonmetropolitan Counties
758,Kansas,Labette6,15,1,0,NaN,0,14,NaN,NaN,20,1,0,KANSAS - Nonmetropolitan Counties



2017


,State,Raw County Data,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson,State/County Type
156,California,San Luis Obispo3,NaN,1,21,16,NaN,1116,400,691,25,5,CALIFORNIA - Metropolitan Counties
166,California,Tulare4,362,13,35,68,246,1883,634,1246,3,0,CALIFORNIA - Metropolitan Counties
470,Idaho,Kootenai5,100,1,25,6,68,NaN,NaN,755,47,6,IDAHO - Metropolitan Counties
589,Indiana,La Porte6,68,0,7,4,57,457,158,251,48,0,INDIANA - Metropolitan Counties
596,Indiana,Vigo6,61,1,8,6,46,879,279,486,114,3,INDIANA - Metropolitan Counties


In [205]:
for year in crimes_dict:
    print(crimes_dict[year].columns)

Index(['State', 'Raw County Data', 'Violent crime',
       'Murder and nonnegligent manslaughter', 'Rape (revised)',
       'Rape (previous def)', 'Robbery', 'Aggravated assault',
       'Property crime', 'Burglary', 'Larceny-theft', 'Motor vehicle theft',
       'Arson', 'State/County Type'],
      dtype='object')
Index(['State', 'Raw County Data', 'Violent crime',
       'Murder and nonnegligent manslaughter', 'Rape (revised)',
       'Rape (previous def)', 'Robbery', 'Aggravated assault',
       'Property crime', 'Burglary', 'Larceny-theft', 'Motor vehicle theft',
       'Arson', 'State/County Type'],
      dtype='object')
Index(['State', 'Raw County Data', 'Violent crime',
       'Murder and nonnegligent manslaughter', 'Rape', 'Robbery',
       'Aggravated assault', 'Property crime', 'Burglary', 'Larceny-theft',
       'Motor vehicle theft', 'Arson', 'State/County Type'],
      dtype='object')


In [206]:
print("Total year's county data with notices:")
for year in crimes_dict:
    crimes_dict[year]['notice num 1'] = crimes_dict[year]['Raw County Data'].str.extract("(\d)")
    crimes_dict[year]['notice num 2'] = crimes_dict[year]['Raw County Data'].str.extract("\d.*(\d)")
    crimes_dict[year]['notice 1'] = crimes_dict[year]['notice num 1'].map(notice_summaries[year])
    crimes_dict[year]['notice 2'] = crimes_dict[year]['notice num 2'].map(notice_summaries[year])
    
    print('              ' ,year)
    print("-"*30)
    display(crimes_dict[year][['notice 1','notice 2']].notnull().sum())
    #display(notice_numbers_table[notice_numbers_table.isnull()])
    #crimes_dict[year][notice_numbers_table.columns] = notice_numbers_table
    #display(crimes_dict[year].loc[crimes_dict[year]['Raw County Data'].str.contains(",")])

print("Counties with multiple notices in a given year:")
for year in crimes_dict:
    display(crimes_dict[year].loc[crimes_dict[year][['notice 1','notice 2']].notnull().all(axis=1)])

Total year's county data with notices:
               2015
------------------------------


notice 1    33
notice 2     1
dtype: int64

               2016
------------------------------


notice 1    36
notice 2     0
dtype: int64

               2017
------------------------------


notice 1    46
notice 2     1
dtype: int64

Counties with multiple notices in a given year:


,State,Raw County Data,Violent crime,Murder and nonnegligent manslaughter,Rape (revised),Rape (previous def),Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson,State/County Type,notice num 1,notice num 2,notice 1,notice 2
2442,Washington,"Snohomish4, 7",476,6,75,NaN,109,286,6373,1918,3526,929,53,WASHINGTON - Metropolitan Counties,4,7,rape classification switched to revised def during year,Reporting Practices not comparable to previous year's data


,State,Raw County Data,Violent crime,Murder and nonnegligent manslaughter,Rape (revised),Rape (previous def),Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson,State/County Type,notice num 1,notice num 2,notice 1,notice 2


,State,Raw County Data,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson,State/County Type,notice num 1,notice num 2,notice 1,notice 2
1635,Ohio,"Lake5,7",NaN,0,NaN,45,NaN,334,61,269,4,1,OHIO - Metropolitan Counties,5,7,Data not recorded due to over reporting,rape def (previous)


In [207]:
#assert crimes_df.loc[crimes_df['Raw County Data'].str.contains("Tulare4")].shape[0] == 1
#crimes_df.loc[crimes_dict[2017]['Raw County Data'].str.contains("Tulare4"),"Motor vehicle theft"] = 679

#crimes_df = crimes_df.loc[~crimes_df['Raw County Data'].str.contains('[1-3]|[5-6]')] #drop all columns with a lack of reporting of statistics
#crimes_df.loc[crimes_df['Raw County Data'].str.contains('4')]

In [208]:
#crimes_df.loc[crimes_df['Raw County Data'].str.contains('[1-3]|[5-9]')]

In [209]:
def rm_all_denotations(place):
    for expr in [' [cC]ity.*',' [bB]orough.*','[pP]arish.*','Census Area.*','[cC]ounty.*','[pP]olice.*']:
        place = re.sub(expr,'',place).strip()
    return place

In [210]:
#crimes_df = crimes_df.loc[~crimes_df['Raw County Data'].str.contains("Police Department")]
for year in crimes_dict:
    crimes_dict[year]['County'] = crimes_dict[year]['Raw County Data'].apply(rm_all_denotations).str.replace("St\.","Saint")

    crimes_dict[year].County = crimes_dict[year]['County'].str.replace("(?<=[a-z]| )\d+.*","").str.strip()
    display(crimes_dict[year].loc[crimes_dict[year]['County'] != crimes_dict[year]['Raw County Data'],['County','Raw County Data']].head())

,County,Raw County Data
128,Saint Francis,St. Francis
136,Butte,Butte4
138,El Dorado,El Dorado4
140,Imperial,Imperial4
142,Kings,Kings4


,County,Raw County Data
24,Saint Clair,St. Clair
126,Saint Francis,St. Francis
165,Tulare,Tulare4
215,Eagle,Eagle5
242,New Castle,New Castle County Police Department


,County,Raw County Data
128,Saint Francis,St. Francis
156,San Luis Obispo,San Luis Obispo3
166,Tulare,Tulare4
244,New Castle,New Castle County Police Department
282,Saint Johns,St. Johns


In [211]:
# merged_cr=crimes_dict[2015].merge(crimes_dict[2016],left_on="Area",right_on="Area")#.iloc[:,3:13].astype(float).corr()
# crime_cols = [col for col in crime_renaming_dict.values() if '(' not in col]
# formatted_crime_cols=merged_cr.columns[merged_cr.columns.str.contains("|".join(crime_cols))]
# merged_cr=merged_cr[formatted_crime_cols]
# merged_cr[formatted_crime_cols].astype(float).corr()

In [212]:
for year in crimes_dict:
    crimes_dict[year]['Area'] = crimes_dict[year]["County"] + ', ' + crimes_dict[year]['State']
    crimes_dict[year]["Loc Abbrev"] = crimes_dict[year]["County"].str[0].str.upper() + ', ' + crimes_dict[year]['State']
    display(crimes_dict[year].head())

,State,Raw County Data,Violent crime,Murder and nonnegligent manslaughter,Rape (revised),Rape (previous def),Robbery,Aggravated assault,Property crime,Burglary,...,Motor vehicle theft,Arson,State/County Type,notice num 1,notice num 2,notice 1,notice 2,County,Area,Loc Abbrev
0,Alabama,Autauga,69,0,13,NaN,6,50,344,111,...,46,NaN,ALABAMA - Metropolitan Counties,NaN,NaN,NaN,NaN,Autauga,"Autauga, Alabama","A, Alabama"
1,Alabama,Baldwin,115,0,9,NaN,33,73,648,225,...,33,NaN,ALABAMA - Metropolitan Counties,NaN,NaN,NaN,NaN,Baldwin,"Baldwin, Alabama","B, Alabama"
2,Alabama,Bibb,7,0,2,NaN,1,4,41,20,...,3,NaN,ALABAMA - Metropolitan Counties,NaN,NaN,NaN,NaN,Bibb,"Bibb, Alabama","B, Alabama"
3,Alabama,Blount,204,5,16,NaN,5,178,832,247,...,82,NaN,ALABAMA - Metropolitan Counties,NaN,NaN,NaN,NaN,Blount,"Blount, Alabama","B, Alabama"
4,Alabama,Calhoun,16,0,4,NaN,1,11,413,181,...,7,NaN,ALABAMA - Metropolitan Counties,NaN,NaN,NaN,NaN,Calhoun,"Calhoun, Alabama","C, Alabama"


,State,Raw County Data,Violent crime,Murder and nonnegligent manslaughter,Rape (revised),Rape (previous def),Robbery,Aggravated assault,Property crime,Burglary,...,Motor vehicle theft,Arson,State/County Type,notice num 1,notice num 2,notice 1,notice 2,County,Area,Loc Abbrev
0,Alabama,Autauga,73,1,12,NaN,8,52,429,146,...,50,NaN,ALABAMA - Metropolitan Counties,NaN,NaN,NaN,NaN,Autauga,"Autauga, Alabama","A, Alabama"
1,Alabama,Baldwin,127,1,5,NaN,23,98,613,229,...,42,NaN,ALABAMA - Metropolitan Counties,NaN,NaN,NaN,NaN,Baldwin,"Baldwin, Alabama","B, Alabama"
2,Alabama,Bibb,0,0,0,NaN,0,0,37,20,...,3,NaN,ALABAMA - Metropolitan Counties,NaN,NaN,NaN,NaN,Bibb,"Bibb, Alabama","B, Alabama"
3,Alabama,Blount,394,1,17,NaN,9,367,867,261,...,105,NaN,ALABAMA - Metropolitan Counties,NaN,NaN,NaN,NaN,Blount,"Blount, Alabama","B, Alabama"
4,Alabama,Calhoun,23,0,7,NaN,5,11,319,137,...,1,NaN,ALABAMA - Metropolitan Counties,NaN,NaN,NaN,NaN,Calhoun,"Calhoun, Alabama","C, Alabama"


,State,Raw County Data,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson,State/County Type,notice num 1,notice num 2,notice 1,notice 2,County,Area,Loc Abbrev
0,Alabama,Autauga,55,2,5,5,43,384,131,206,47,NaN,ALABAMA - Metropolitan Counties,NaN,NaN,NaN,NaN,Autauga,"Autauga, Alabama","A, Alabama"
1,Alabama,Baldwin,163,1,9,18,135,737,226,442,69,NaN,ALABAMA - Metropolitan Counties,NaN,NaN,NaN,NaN,Baldwin,"Baldwin, Alabama","B, Alabama"
2,Alabama,Bibb,3,0,1,1,1,69,41,24,4,NaN,ALABAMA - Metropolitan Counties,NaN,NaN,NaN,NaN,Bibb,"Bibb, Alabama","B, Alabama"
3,Alabama,Blount,313,1,14,5,293,812,198,502,112,NaN,ALABAMA - Metropolitan Counties,NaN,NaN,NaN,NaN,Blount,"Blount, Alabama","B, Alabama"
4,Alabama,Calhoun,18,0,7,7,4,211,93,117,1,NaN,ALABAMA - Metropolitan Counties,NaN,NaN,NaN,NaN,Calhoun,"Calhoun, Alabama","C, Alabama"


In [213]:
for year in crimes_dict:
    print("year {}:".format(year),crimes_dict[year].shape)
crimes_dict[2017].Area.isin(crimes_dict[2015].Area).sum()

year 2015: (2587, 21)
year 2016: (2612, 21)
year 2017: (2517, 20)


2343

In [214]:
#county_list = crimes_dict[year].County.tolist()
#new_df = pd.merge(A_df, B_df,  how='left', left_on=['A_c1','c2'], right_on = ['B_c1','c2'])
police_dept_counties_dict={}
police_dept_areas_dict={}
for year in crimes_dict:
    police_dept_counties_dict[year]=crimes_dict[year].loc[crimes_dict[year]["Raw County Data"].str.contains("Police Department"),['State','County']]

    police_dept_areas_dict[year] = pd.merge(crimes_dict[year],police_dept_counties_dict[year],
                                            left_on = ['State','County'],
                                            right_on=['State','County']).sort_values(by=['State','County'])
# from what can be seen, it is quite possible that some entries were segmented into two separate listings.
# within this analysis, these entries will be combined
    display(police_dept_areas_dict[year])
#police_dept_areas


,State,Raw County Data,Violent crime,Murder and nonnegligent manslaughter,Rape (revised),Rape (previous def),Robbery,Aggravated assault,Property crime,Burglary,...,Motor vehicle theft,Arson,State/County Type,notice num 1,notice num 2,notice 1,notice 2,County,Area,Loc Abbrev
0,Delaware,New Castle County Police Department,1008,12,69,NaN,218,709,4480,1112,...,352,3,DELAWARE - Metropolitan Counties,NaN,NaN,NaN,NaN,New Castle,"New Castle, Delaware","N, Delaware"
1,Georgia,Clayton,3,0,NaN,0,0,3,10,0,...,3,0,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,Clayton,"Clayton, Georgia","C, Georgia"
2,Georgia,Clayton County Police Department,1202,18,NaN,88,550,546,8275,2513,...,1158,7,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,Clayton,"Clayton, Georgia","C, Georgia"
3,Georgia,Cobb,113,0,NaN,1,4,108,34,5,...,0,0,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,Cobb,"Cobb, Georgia","C, Georgia"
4,Georgia,Cobb County Police Department,1208,17,NaN,107,445,639,11982,2410,...,870,37,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,Cobb,"Cobb, Georgia","C, Georgia"
5,Georgia,DeKalb,0,0,NaN,0,0,0,0,0,...,0,NaN,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,DeKalb,"DeKalb, Georgia","D, Georgia"
6,Georgia,DeKalb County Police Department,3583,70,NaN,143,1820,1550,24255,6823,...,3713,129,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,DeKalb,"DeKalb, Georgia","D, Georgia"
7,Georgia,Dougherty,1,0,NaN,0,0,1,85,3,...,1,0,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,Dougherty,"Dougherty, Georgia","D, Georgia"
8,Georgia,Dougherty County Police Department,27,0,NaN,2,7,18,306,103,...,18,1,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,Dougherty,"Dougherty, Georgia","D, Georgia"
9,Georgia,Floyd,31,2,NaN,2,10,17,43,8,...,3,NaN,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,Floyd,"Floyd, Georgia","F, Georgia"


,State,Raw County Data,Violent crime,Murder and nonnegligent manslaughter,Rape (revised),Rape (previous def),Robbery,Aggravated assault,Property crime,Burglary,...,Motor vehicle theft,Arson,State/County Type,notice num 1,notice num 2,notice 1,notice 2,County,Area,Loc Abbrev
0,Delaware,New Castle County Police Department,969,14,44,NaN,212,699,5485,1318,...,500,1,DELAWARE - Metropolitan Counties,NaN,NaN,NaN,NaN,New Castle,"New Castle, Delaware","N, Delaware"
1,Georgia,Clayton,6,0,NaN,0,1,5,27,5,...,3,0,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,Clayton,"Clayton, Georgia","C, Georgia"
2,Georgia,Clayton County Police Department,1504,39,NaN,162,603,700,8686,2348,...,1257,16,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,Clayton,"Clayton, Georgia","C, Georgia"
3,Georgia,Cobb,79,0,NaN,5,8,66,39,8,...,0,0,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,Cobb,"Cobb, Georgia","C, Georgia"
4,Georgia,Cobb County Police Department,1262,16,NaN,88,461,697,11609,2184,...,935,33,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,Cobb,"Cobb, Georgia","C, Georgia"
5,Georgia,DeKalb,0,0,NaN,0,0,0,0,0,...,0,0,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,DeKalb,"DeKalb, Georgia","D, Georgia"
6,Georgia,DeKalb County Police Department,3452,76,NaN,138,1595,1643,23887,6024,...,3791,141,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,DeKalb,"DeKalb, Georgia","D, Georgia"
7,Georgia,Dougherty,1,0,NaN,0,0,1,171,0,...,0,0,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,Dougherty,"Dougherty, Georgia","D, Georgia"
8,Georgia,Dougherty County Police Department,44,0,NaN,3,8,33,308,107,...,18,0,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,Dougherty,"Dougherty, Georgia","D, Georgia"
9,Georgia,Floyd,16,0,NaN,1,4,11,81,21,...,4,0,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,Floyd,"Floyd, Georgia","F, Georgia"


,State,Raw County Data,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson,State/County Type,notice num 1,notice num 2,notice 1,notice 2,County,Area,Loc Abbrev
0,Delaware,New Castle County Police Department,933,5,50,161,717,4308,951,2968,389,4,DELAWARE - Metropolitan Counties,NaN,NaN,NaN,NaN,New Castle,"New Castle, Delaware","N, Delaware"
1,Georgia,Clayton,6,0,0,0,6,10,0,8,2,0,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,Clayton,"Clayton, Georgia","C, Georgia"
2,Georgia,Clayton County Police Department,1543,35,183,506,819,8711,2078,5210,1423,12,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,Clayton,"Clayton, Georgia","C, Georgia"
3,Georgia,Cobb,7,0,2,4,1,26,2,24,0,0,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,Cobb,"Cobb, Georgia","C, Georgia"
4,Georgia,Cobb County Police Department,1221,19,116,408,678,11899,1846,9004,1049,30,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,Cobb,"Cobb, Georgia","C, Georgia"
5,Georgia,DeKalb,0,0,0,0,0,0,0,0,0,0,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,DeKalb,"DeKalb, Georgia","D, Georgia"
6,Georgia,DeKalb County Police Department,3184,89,112,1249,1734,23246,4994,15029,3223,122,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,DeKalb,"DeKalb, Georgia","D, Georgia"
7,Georgia,Dougherty,1,0,0,0,1,98,0,97,1,0,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,Dougherty,"Dougherty, Georgia","D, Georgia"
8,Georgia,Dougherty County Police Department,35,0,5,7,23,322,95,202,25,1,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,Dougherty,"Dougherty, Georgia","D, Georgia"
9,Georgia,Fulton,11,0,0,1,10,7,0,7,0,0,GEORGIA - Metropolitan Counties,NaN,NaN,NaN,NaN,Fulton,"Fulton, Georgia","F, Georgia"


In [215]:
county_counts_dict={}
for year in crimes_dict:
    county_counts_dict[year] = (police_dept_areas_dict[year]
                     .groupby("Area")['County']
                     .value_counts()
                     .rename(index='Frequency')
                     .to_frame()
                     .reset_index()
                    )
    display(county_counts_dict[year])

,Area,County,Frequency
0,"Albemarle, Virginia",Albemarle,1
1,"Allegheny, Pennsylvania",Allegheny,2
2,"Arlington, Virginia",Arlington,1
3,"Bergen, New Jersey",Bergen,2
4,"Camden, New Jersey",Camden,2
5,"Campbell, Kentucky",Campbell,2
6,"Chesterfield, Virginia",Chesterfield,1
7,"Clayton, Georgia",Clayton,2
8,"Cobb, Georgia",Cobb,2
9,"DeKalb, Georgia",DeKalb,2


,Area,County,Frequency
0,"Albemarle, Virginia",Albemarle,1
1,"Anne Arundel, Maryland",Anne Arundel,2
2,"Arlington, Virginia",Arlington,1
3,"Baltimore, Maryland",Baltimore,2
4,"Camden, New Jersey",Camden,2
5,"Campbell, Kentucky",Campbell,2
6,"Chesterfield, Virginia",Chesterfield,1
7,"Clayton, Georgia",Clayton,2
8,"Cobb, Georgia",Cobb,2
9,"DeKalb, Georgia",DeKalb,2


,Area,County,Frequency
0,"Albemarle, Virginia",Albemarle,1
1,"Allegheny, Pennsylvania",Allegheny,2
2,"Anne Arundel, Maryland",Anne Arundel,2
3,"Arlington, Virginia",Arlington,1
4,"Baltimore, Maryland",Baltimore,2
5,"Campbell, Kentucky",Campbell,2
6,"Chesterfield, Virginia",Chesterfield,1
7,"Clayton, Georgia",Clayton,2
8,"Cobb, Georgia",Cobb,2
9,"DeKalb, Georgia",DeKalb,2


In [216]:
county_counts_dict

{2015:                          Area           County  Frequency
 0         Albemarle, Virginia        Albemarle          1
 1     Allegheny, Pennsylvania        Allegheny          2
 2         Arlington, Virginia        Arlington          1
 3          Bergen, New Jersey           Bergen          2
 4          Camden, New Jersey           Camden          2
 5          Campbell, Kentucky         Campbell          2
 6      Chesterfield, Virginia     Chesterfield          1
 7            Clayton, Georgia          Clayton          2
 8               Cobb, Georgia             Cobb          2
 9             DeKalb, Georgia           DeKalb          2
 10         Dougherty, Georgia        Dougherty          2
 11          Fairfax, Virginia          Fairfax          1
 12             Floyd, Georgia            Floyd          2
 13            Fulton, Georgia           Fulton          2
 14             Glynn, Georgia            Glynn          1
 15          Gwinnett, Georgia         Gwinnett   

In [217]:
unique_police_counties_dict={}
for year in county_counts_dict:
    unique_police_counties_dict[year] = county_counts_dict[year].query('Frequency == 1')
    
    print('             ',year)
    print("-"*37)
    display(unique_police_counties_dict[year])
    print("Shape:",unique_police_counties_dict[year].shape,'\n\n')

              2015
-------------------------------------


,Area,County,Frequency
0,"Albemarle, Virginia",Albemarle,1
2,"Arlington, Virginia",Arlington,1
6,"Chesterfield, Virginia",Chesterfield,1
11,"Fairfax, Virginia",Fairfax,1
14,"Glynn, Georgia",Glynn,1
15,"Gwinnett, Georgia",Gwinnett,1
16,"Henrico, Virginia",Henrico,1
20,"James, Virginia",James,1
22,"Maui, Hawaii",Maui,1
25,"New Castle, Delaware",New Castle,1


Shape: (16, 3) 


              2016
-------------------------------------


,Area,County,Frequency
0,"Albemarle, Virginia",Albemarle,1
2,"Arlington, Virginia",Arlington,1
6,"Chesterfield, Virginia",Chesterfield,1
11,"Fairfax, Virginia",Fairfax,1
15,"Gwinnett, Georgia",Gwinnett,1
16,"Hawaii, Hawaii",Hawaii,1
17,"Henrico, Virginia",Henrico,1
21,"James, Virginia",James,1
22,"Kauai, Hawaii",Kauai,1
24,"Maui, Hawaii",Maui,1


Shape: (17, 3) 


              2017
-------------------------------------


,Area,County,Frequency
0,"Albemarle, Virginia",Albemarle,1
3,"Arlington, Virginia",Arlington,1
6,"Chesterfield, Virginia",Chesterfield,1
11,"Fairfax, Virginia",Fairfax,1
13,"Gaston, North Carolina",Gaston,1
15,"Gwinnett, Georgia",Gwinnett,1
16,"Hawaii, Hawaii",Hawaii,1
17,"Henrico, Virginia",Henrico,1
21,"James, Virginia",James,1
22,"Kauai, Hawaii",Kauai,1


Shape: (19, 3) 




In [218]:
#counties_police_dept_only = (police_dept_areas.County.value_counts() == 1).index

In [219]:
crime_categories_dict = {}
county_data_dict = {}
true_county_sums_dict= {}

for year in crimes_dict:
    crime_categories_dict[year]=crimes_dict[year].columns[(crimes_dict[year].columns.str.contains("County|State|Area|year|notice") == False)]
    #print('             ',year)
    #print("-"*37)
    #print(*crime_categories_dict[year].tolist(),sep='\n')
    county_data_dict[year]=crimes_dict[year].columns[~np.isin(crimes_dict[year].columns,crime_categories_dict[year])]
    #print(*county_data_dict[year].tolist(),sep='\n')
    #def sum_county_crime(county):

    #police_dept_counties.loc[~police_dept_counties.County.isin(police_dept_areas_dict.County)]
    #help(pd.merge)
    true_county_sums_dict[year]=police_dept_areas_dict[year].groupby('Area',as_index=False)[crime_categories_dict[year]].sum()
    display(true_county_sums_dict[year].head())

,Area,Violent crime,Murder and nonnegligent manslaughter,Rape (revised),Rape (previous def),Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson,Loc Abbrev
0,"Albemarle, Virginia",103,0,44,0,13,46,1348,191,1107,50,9,"A, Virginia"
1,"Allegheny, Pennsylvania",57,0,3,0,4,50,187,1,159,27,98,"A, PennsylvaniaA, Pennsylvania"
2,"Arlington, Virginia",317,1,37,0,114,165,3330,204,2962,164,10,"A, Virginia"
3,"Bergen, New Jersey",7,0,0,0,0,7,57,3,53,1,0,"B, New JerseyB, New Jersey"
4,"Camden, New Jersey",1524,32,75,0,483,934,2804,675,1791,338,87,"C, New JerseyC, New Jersey"


,Area,Violent crime,Murder and nonnegligent manslaughter,Rape (revised),Rape (previous def),Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson,Loc Abbrev
0,"Albemarle, Virginia",88,1,29,0,21,37,1271,145,1061,65,8,"A, Virginia"
1,"Anne Arundel, Maryland",2053,14,143,0,492,1404,11642,1767,9302,573,33,"A, MarylandA, Maryland"
2,"Arlington, Virginia",363,1,56,0,103,203,3252,170,2912,170,5,"A, Virginia"
3,"Baltimore, Maryland",4397,35,318,0,1497,2547,23344,3681,17599,2064,163,"B, MarylandB, Maryland"
4,"Camden, New Jersey",1583,47,88,0,428,1020,2676,650,1522,504,94,"C, New JerseyC, New Jersey"


,Area,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson,Loc Abbrev
0,"Albemarle, Virginia",100,5,44,11,40,1261,144,1038,79,4,"A, Virginia"
1,"Allegheny, Pennsylvania",178,3,33,13,129,181,24,144,13,10,"A, PennsylvaniaA, Pennsylvania"
2,"Anne Arundel, Maryland",2023,13,156,566,1288,10813,1593,8598,622,0,"A, MarylandA, Maryland"
3,"Arlington, Virginia",359,5,71,73,210,2913,157,2610,146,3,"A, Virginia"
4,"Baltimore, Maryland",5037,35,323,1533,3146,23155,3114,18023,2018,206,"B, MarylandB, Maryland"


In [220]:
for year in crimes_dict:
    assert (crimes_dict[year].loc[crimes_dict[year]['Raw County Data'].str.contains('Police Department'),'Raw County Data'].value_counts()>1).sum() == 0, 'At least 1 county contains more than one police department reporting agency'

In [221]:
for year in crimes_dict:
    is_unique_dept = crimes_dict[year]['Area'].isin(unique_police_counties_dict[year]['Area'].values)
    crimes_dict[year] = crimes_dict[year].loc[(~crimes_dict[year]['Raw County Data'].str.contains("Police")) | (is_unique_dept)]

In [222]:
for year in crimes_dict:
    print(crimes_dict[year]['Raw County Data'].str.contains("Police").sum())

16
17
19


In [223]:
for year in crimes_dict:
    assert crimes_dict[year].Area.duplicated().sum() == 0, "At least one area name contains more than 1 entry"

In [224]:
def update_crime_counts(area_crime_data,year):
    
    if area_crime_data.Area in true_county_sums_dict[year].Area.values:
        crimes=crime_categories_dict[year].tolist()
        area_crime_data[crimes] = true_county_sums_dict[year].loc[true_county_sums_dict[year].Area == area_crime_data.Area, crimes].squeeze()
    return area_crime_data


print("Total Updated Values:")
for year in crimes_dict:
    crime_data_copy = crimes_dict[year].copy()
    crimes_dict[year] = crimes_dict[year].apply(update_crime_counts,year=year,axis=1)
    print('                ',year)
    print("-"*41)
    #display(crimes_dict[year].head())
    display((crimes_dict[year].apply(update_crime_counts,year=year,axis=1).fillna("") != crime_data_copy.fillna("")).sum())
    print('\n\n')

Total Updated Values:
                 2015
-----------------------------------------


State                                    0
Raw County Data                          0
Violent crime                           20
Murder and nonnegligent manslaughter    14
Rape (revised)                          21
Rape (previous def)                     35
Robbery                                 19
Aggravated assault                      20
Property crime                          20
Burglary                                19
Larceny-theft                           20
Motor vehicle theft                     20
Arson                                   15
State/County Type                        0
notice num 1                             0
notice num 2                             0
notice 1                                 0
notice 2                                 0
County                                   0
Area                                     0
Loc Abbrev                              22
dtype: int64




                 2016
-----------------------------------------


State                                    0
Raw County Data                          0
Violent crime                           20
Murder and nonnegligent manslaughter    16
Rape (revised)                          21
Rape (previous def)                     36
Robbery                                 19
Aggravated assault                      20
Property crime                          20
Burglary                                20
Larceny-theft                           20
Motor vehicle theft                     20
Arson                                   18
State/County Type                        0
notice num 1                             0
notice num 2                             0
notice 1                                 0
notice 2                                 0
County                                   0
Area                                     0
Loc Abbrev                              21
dtype: int64




                 2017
-----------------------------------------


State                                    0
Raw County Data                          0
Violent crime                           20
Murder and nonnegligent manslaughter    17
Rape                                    20
Robbery                                 18
Aggravated assault                      19
Property crime                          19
Burglary                                19
Larceny-theft                           19
Motor vehicle theft                     19
Arson                                   16
State/County Type                        0
notice num 1                             0
notice num 2                             0
notice 1                                 0
notice 2                                 0
County                                   0
Area                                     0
Loc Abbrev                              19
dtype: int64

In [225]:
# some counties in some datasets have more than 1 entry with the same name, but with different county/city names.
#the following code checks for all such occurences:

for year in crimes_dict:
    county_name_freq=crimes_dict[year].groupby("State")['County'].value_counts()
    assert (county_name_freq > 1).sum() == 0

In [226]:
#Ensure that we have truly replaced all instances where the word, county appears.
for year in crimes_dict:
    assert crimes_dict[year].County.str.contains("County|St\.").sum() == 0

In [227]:
def find_unique_locations(dataset,comparison_dataset):
    """This function is soley meant to test whether the leftmost dataset contains any locations that are not
    present in the comparison dataset."""
    return sorted(set(dataset.Area).difference(comparison_dataset.Area))

crime_areas=pd.concat([crimes_dict[current_year][["Area","County"]].assign(year=current_year) for current_year in crimes_dict])
crime_areas['County Letter']=crime_areas['Area'].apply(lambda area: area[0])
crime_areas['State'] = crime_areas.apply(lambda area_data: area_data['Area'].replace(area_data['County'] + ', ',''),axis=1).str.strip()
crime_areas['Loc Abbrev'] = crime_areas['County Letter'] + ', ' + crime_areas['State']
crime_areas=pd.concat([crime_areas,pd.get_dummies(crime_areas.year)],axis=1)


def sum_area_data(area_data):
    return area_data.sum(axis=0)

crime_areas[[2015,2016,2017]] = crime_areas.groupby("Area")[[2015,2016,2017]].transform(np.sum,axis=0)
crime_areas=crime_areas.drop('year',axis=1)
crime_areas['Report Count']=crime_areas[[2015,2016,2017]].sum(axis=1)
crime_areas=crime_areas.drop_duplicates()
crime_areas

,Area,County,County Letter,State,Loc Abbrev,2015,2016,2017,Report Count
0,"Autauga, Alabama",Autauga,A,Alabama,"A, Alabama",1,1,1,3
1,"Baldwin, Alabama",Baldwin,B,Alabama,"B, Alabama",1,1,1,3
2,"Bibb, Alabama",Bibb,B,Alabama,"B, Alabama",1,1,1,3
3,"Blount, Alabama",Blount,B,Alabama,"B, Alabama",1,1,1,3
4,"Calhoun, Alabama",Calhoun,C,Alabama,"C, Alabama",1,1,1,3
...,...,...,...,...,...,...,...,...,...
2250,"Essex, Vermont",Essex,E,Vermont,"E, Vermont",0,0,1,1
2252,"Orange, Vermont",Orange,O,Vermont,"O, Vermont",0,0,1,1
2255,"Washington, Vermont",Washington,W,Vermont,"W, Vermont",0,0,1,1
2257,"Windsor, Vermont",Windsor,W,Vermont,"W, Vermont",0,0,1,1


In [228]:
crime_areas["Regex Area Name"] = crime_areas['Area']

vowles='a|e|i|o|u'

def modify_crime_county_names(area_data):
    if area_data['Report Count'] == 3:
        return area_data
    else:
        county = area_data['County']
        regex_county_name = re.sub(vowles,'.*',county)
        area_data['Regex Area Name'] = area_data['Regex Area Name'].replace(county,regex_county_name).lower()
        return area_data
crime_areas=crime_areas.apply(modify_crime_county_names,axis=1)    
crime_areas["Updated Counts"]=crime_areas.groupby("Regex Area Name",as_index=False)['Report Count'].transform(np.sum)

crime_areas.loc[crime_areas['Updated Counts'] != crime_areas['Report Count']]

,Area,County,County Letter,State,Loc Abbrev,2015,2016,2017,Report Count,Regex Area Name,Updated Counts
577,"Bartholomew, Indiana",Bartholomew,B,Indiana,"B, Indiana",1,0,1,2,"b.*rth.*l.*m.*w, indiana",3
1130,"Lac Qui Parle, Minnesota",Lac Qui Parle,L,Minnesota,"L, Minnesota",1,0,0,1,"l.*c q.*.* p.*rl.*, minnesota",3
589,"Bartholemew, Indiana",Bartholemew,B,Indiana,"B, Indiana",0,1,0,1,"b.*rth.*l.*m.*w, indiana",3
1139,"Lac qui Parle, Minnesota",Lac qui Parle,L,Minnesota,"L, Minnesota",0,1,1,2,"l.*c q.*.* p.*rl.*, minnesota",3


In [229]:
#def filter_potential_matches(area_data):

unfully_reported_areas=crime_areas.loc[crime_areas['Report Count']<3]
loc_abbrev_counts = unfully_reported_areas['Loc Abbrev'].value_counts()
loc_abbrev_counts
#potential_same_area = unfully_reported_areas['Loc Abbrev'].map(loc_abbrev_counts) > 1

#sorted_area_name_df=unfully_reported_areas.loc[potentially_same_areas.values].sort_values(by=['Loc Abbrev','County'])

#pd.options.display.max_rows=370
#display(sorted_area_name_df)
#pd.options.display.max_rows=100


S, Nebraska          7
S, Kansas            6
W, Georgia           6
C, Nebraska          6
B, Georgia           5
                    ..
C, South Carolina    1
B, Minnesota         1
M, Alabama           1
G, Utah              1
F, Kentucky          1
Name: Loc Abbrev, Length: 314, dtype: int64

In [230]:
relevant_columns = [var for var in unfully_reported_areas.columns if not re.search('\d+',str(var)) and var not in ["Regex Area Name","Updated Counts","Report Count","State","County Letter"]]

# county_comparison_df=unfully_reported_areas.merge(unfully_reported_areas[relevant_columns],
#                                                   left_on="Loc Abbrev",right_on='Loc Abbrev',
#                                                   suffixes=('1','2')).query("Area1!=Area2").drop_duplicates().reset_index(drop=True)

# duplicated_comparisons=pd.DataFrame((np.sort(county_comparison_df[['Area1','Area2']],axis=1))).duplicated()
# county_comparison_df=county_comparison_df.loc[~duplicated_comparisons.values]
# ld=county_comparison_df.apply(lambda area_data: levenshtein_distance(area_data.County1,area_data.County2),axis=1)
# similar_names_df = county_comparison_df.loc[(ld < 3)]
# similar_names_df

def unique_combinational_merge(df1,df2,comparison_col1,comparison_col2,left_on=None,right_on=None,filter_comparisons=True,suffixes=("_1","_2")):
    try:
        assert comparison_col1 != comparison_col2, "Columns contain the same name"
    
    except AssertionError:
        comparison_col1 = str(comparison_col1) + suffixes[0]
        comparison_col2 = str(comparison_col2) + suffixes[1]
        
        assert comparison_col1 not in df1.columns, "{} column already found in the columns of leftmost dataframe.\n try renaming the columns before merging.".format(comparison_col1)
        assert comparison_col2 not in df1.columns, "{} column already found in the columns of rightmost dataframe.\n try renaming the columns before merging.".format(comparison_col2)
    
    product_df=df1.merge(df2,left_on=left_on,right_on=right_on,suffixes=suffixes).dropna()#.query(comparison_col1+'!='+comparison_col2)
    if filter_comparisons:
        same_row_indices = product_df.loc[product_df[comparison_col1] == product_df[comparison_col2]].index.tolist()
        product_df=product_df.drop(same_row_indices).reset_index()

        
    product_df = product_df.drop_duplicates().dropna().reset_index(drop=True)
    duplicated_combinations=pd.DataFrame(np.sort(product_df[[comparison_col1,comparison_col2]].values,axis=1)).duplicated()
    duplicated_idx = duplicated_combinations.loc[duplicated_combinations].index
    product_df=product_df.drop(duplicated_idx)
    return product_df
# potential issues: Saint John (the Baptist), Lousiana
# Saint Mary('s) Maryland

# To determine which counties were contained in multiple data sets yet spelled differently, all
# rows that started with the same county name rom the same state were merged together.
# Merging the final data set of crimes data will merge on exact same county names within each state
# but will not account for county names that are similarly yet not identically spelled.
# Merging on county letter + state followed by the use of the Levenshtein distance will, in and of itself,
# allow for an ease of identification of what county names are spelled slightly differently across data sets.
# The Levenshtein distance is a measure of the difference between two strings. It counts how many steps are needed
# in order for one string to equal the other. For every letter deletion, replacement, and addition the count 
# for the number of transformations needed for one string to equal another string goes up by 1. Therefore, higher
# counts denote larger differences between two strings.

county_comparison_df=unique_combinational_merge(
    unfully_reported_areas,unfully_reported_areas[relevant_columns],
    comparison_col1='Area',
    comparison_col2='Area',
    left_on="Loc Abbrev",right_on='Loc Abbrev'
)
county_comparison_df

,index,Area_1,County_1,County Letter,State,Loc Abbrev,2015,2016,2017,Report Count,Regex Area Name,Updated Counts,Area_2,County_2
0,3,"Choctaw, Alabama",Choctaw,C,Alabama,"C, Alabama",1,0,1,2,"ch.*ct.*w, alabama",2,"Clay, Alabama",Clay
1,4,"Choctaw, Alabama",Choctaw,C,Alabama,"C, Alabama",1,0,1,2,"ch.*ct.*w, alabama",2,"Crenshaw, Alabama",Crenshaw
2,5,"Choctaw, Alabama",Choctaw,C,Alabama,"C, Alabama",1,0,1,2,"ch.*ct.*w, alabama",2,"Colbert, Alabama",Colbert
4,8,"Clay, Alabama",Clay,C,Alabama,"C, Alabama",1,1,0,2,"cl.*y, alabama",2,"Crenshaw, Alabama",Crenshaw
5,9,"Clay, Alabama",Clay,C,Alabama,"C, Alabama",1,1,0,2,"cl.*y, alabama",2,"Colbert, Alabama",Colbert
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,1231,"Stokes, North Carolina",Stokes,S,North Carolina,"S, North Carolina",0,1,1,2,"st.*k.*s, north carolina",2,"Scotland, North Carolina",Scotland
739,1234,"Sampson, North Carolina",Sampson,S,North Carolina,"S, North Carolina",0,1,0,1,"s.*mps.*n, north carolina",1,"Scotland, North Carolina",Scotland
742,1242,"Lane, Oregon",Lane,L,Oregon,"L, Oregon",0,1,1,2,"l.*n.*, oregon",2,"Lincoln, Oregon",Lincoln
744,1258,"Pleasants, West Virginia",Pleasants,P,West Virginia,"P, West Virginia",0,1,0,1,"pl.*.*s.*nts, west virginia",1,"Pocahontas, West Virginia",Pocahontas


In [231]:
ld=county_comparison_df.apply(lambda area_data: levenshtein_distance(area_data.County_1,area_data.County_2),axis=1)
similar_names_df = county_comparison_df.loc[(ld < 3)]
similar_names_df


,index,Area_1,County_1,County Letter,State,Loc Abbrev,2015,2016,2017,Report Count,Regex Area Name,Updated Counts,Area_2,County_2
36,71,"Bartow, Georgia",Bartow,B,Georgia,"B, Georgia",1,1,0,2,"b.*rt.*w, georgia",2,"Barrow, Georgia",Barrow
130,209,"Bartholomew, Indiana",Bartholomew,B,Indiana,"B, Indiana",1,0,1,2,"b.*rth.*l.*m.*w, indiana",3,"Bartholemew, Indiana",Bartholemew
256,400,"Lane, Kansas",Lane,L,Kansas,"L, Kansas",1,0,1,2,"l.*n.*, kansas",2,"Linn, Kansas",Linn
265,411,"Linn, Kansas",Linn,L,Kansas,"L, Kansas",0,1,0,1,"l.*nn, kansas",1,"Lyon, Kansas",Lyon
338,529,"Lac Qui Parle, Minnesota",Lac Qui Parle,L,Minnesota,"L, Minnesota",1,0,0,1,"l.*c q.*.* p.*rl.*, minnesota",3,"Lac qui Parle, Minnesota",Lac qui Parle
429,655,"Sheridan, Nebraska",Sheridan,S,Nebraska,"S, Nebraska",1,1,0,2,"sh.*r.*d.*n, nebraska",2,"Sherman, Nebraska",Sherman
492,752,"Story, Nevada",Story,S,Nevada,"S, Nevada",1,0,0,1,"st.*ry, nevada",1,"Storey, Nevada",Storey
498,767,"Duchess, New York",Duchess,D,New York,"D, New York",1,0,0,1,"d.*ch.*ss, new york",1,"Dutchess, New York",Dutchess
623,999,"Martin, Texas",Martin,M,Texas,"M, Texas",1,1,0,2,"m.*rt.*n, texas",2,"Marion, Texas",Marion


In [232]:
area_names=similar_names_df.loc[similar_names_df.Area_2.str.contains("Barthol|Lac|Storey|Dutchess"),["Area_1",'Area_2']]

#columns to rearrange:
dutchess=area_names.loc[area_names["Area_2"].str.contains("Dutch.*, New York"),["Area_1","Area_2"]].values.flatten().tolist()
area_names.loc[area_names["Area_2"].str.contains("Dutch.*, New York"),["Area_1","Area_2"]] = dutchess[::-1]

area_renaming_dict=area_names.set_index("Area_2")['Area_1'].squeeze().to_dict()
area_renaming_dict

{'Bartholemew, Indiana': 'Bartholomew, Indiana',
 'Lac qui Parle, Minnesota': 'Lac Qui Parle, Minnesota',
 'Storey, Nevada': 'Story, Nevada',
 'Duchess, New York': 'Dutchess, New York'}

In [233]:
for year in crimes_dict:
    for mispelled_area, area_name in area_renaming_dict.items():
        if mispelled_area in crimes_dict[year].Area.values:
            area_idx=crimes_dict[year].loc[crimes_dict[year].Area == mispelled_area].index
            assert len(area_idx) == 1
            county = crime_areas.loc[crime_areas['Area']==area_name,'County'].values[0]
            
            print("Modifying ({})".format(year),crimes_dict[year].loc[area_idx,"Area"].values[0], ' to', area_name)
            previous_county_name=crimes_dict[year].loc[area_idx,"County"].values[0]
            crimes_dict[year].loc[area_idx,"Raw County Data"] = crimes_dict[year].loc[area_idx,"Raw County Data"].str.replace(previous_county_name,county,regex=False)
            crimes_dict[year].loc[area_idx,['County',"Area"]] = [county, area_name]
            print("Success")

Modifying (2015) Duchess, New York  to Dutchess, New York
Success
Modifying (2016) Bartholemew, Indiana  to Bartholomew, Indiana
Success
Modifying (2016) Lac qui Parle, Minnesota  to Lac Qui Parle, Minnesota
Success
Modifying (2016) Storey, Nevada  to Story, Nevada
Success
Modifying (2017) Lac qui Parle, Minnesota  to Lac Qui Parle, Minnesota
Success
Modifying (2017) Storey, Nevada  to Story, Nevada
Success


In [234]:
crime_areas.loc[crime_areas.Area.str.contains("Augusta|Richmond")]

,Area,County,County Letter,State,Loc Abbrev,2015,2016,2017,Report Count,Regex Area Name,Updated Counts
1597,"Richmond, North Carolina",Richmond,R,North Carolina,"R, North Carolina",1,1,1,3,"Richmond, North Carolina",3
2339,"Augusta, Virginia",Augusta,A,Virginia,"A, Virginia",1,1,1,3,"Augusta, Virginia",3
2418,"Richmond, Virginia",Richmond,R,Virginia,"R, Virginia",1,1,1,3,"Richmond, Virginia",3
308,"Augusta-Richmond, Georgia",Augusta-Richmond,A,Georgia,"A, Georgia",0,1,1,2,"a.*g.*st.*-r.*chm.*nd, georgia",2


In [235]:
from functools import reduce,partial
def format_data_for_merging(data,year,is_column_host,static_col_names = ['State','County','Area',"Loc Abbrev"]):
    
    if is_column_host:
        dynamic_col_names = data[~np.isin(data,static_col_names)]
        renaming_dict = {colname: str(colname) +" ({})".format(year) for colname in dynamic_col_names if colname not in static_col_names}
        return data.rename(columns = renaming_dict)
    else:
        dynamic_col_names = data[~np.isin(data,static_col_names)]
        renaming_dict = {colname: str(colname) + " ({})".format(year) for colname in dynamic_col_names if colname not in static_col_names}
        return data.drop(static_col_names[:2],axis=1).rename(columns=renaming_dict)
# the following command will be used to merge each crime data set using a full join
counties_merge = partial(pd.merge,how='outer',left_on='Area',right_on='Area')

# the following command will be used later to merge free market rent data with the then existent merged data set
counties_merge_left = partial(pd.merge,how='left',left_on='Area',right_on='Area')

# merging all 3 crimes data sets:
# format_data_for_merging adds years next to the name of the crime.
# if not "is_column_host" is true, then drop the static_col_names, since the pre-exist in the data set already.
crimes_df=reduce(counties_merge,[format_data_for_merging(crimes_dict[year],year,is_column_host = year == 2015) for year in [2015,2016,2017]])

print("Preview of the merged data:")
display(crimes_df.head())
crimes_df.shape

Preview of the merged data:


,State,Raw County Data (2015),Violent crime (2015),Murder and nonnegligent manslaughter (2015),Rape (revised) (2015),Rape (previous def) (2015),Robbery (2015),Aggravated assault (2015),Property crime (2015),Burglary (2015),...,Burglary (2017),Larceny-theft (2017),Motor vehicle theft (2017),Arson (2017),State/County Type (2017),notice num 1 (2017),notice num 2 (2017),notice 1 (2017),notice 2 (2017),Loc Abbrev
0,Alabama,Autauga,69.0,0.0,13.0,NaN,6.0,50.0,344.0,111.0,...,131.0,206.0,47.0,NaN,ALABAMA - Metropolitan Counties,NaN,NaN,NaN,NaN,"A, Alabama"
1,Alabama,Baldwin,115.0,0.0,9.0,NaN,33.0,73.0,648.0,225.0,...,226.0,442.0,69.0,NaN,ALABAMA - Metropolitan Counties,NaN,NaN,NaN,NaN,"B, Alabama"
2,Alabama,Bibb,7.0,0.0,2.0,NaN,1.0,4.0,41.0,20.0,...,41.0,24.0,4.0,NaN,ALABAMA - Metropolitan Counties,NaN,NaN,NaN,NaN,"B, Alabama"
3,Alabama,Blount,204.0,5.0,16.0,NaN,5.0,178.0,832.0,247.0,...,198.0,502.0,112.0,NaN,ALABAMA - Metropolitan Counties,NaN,NaN,NaN,NaN,"B, Alabama"
4,Alabama,Calhoun,16.0,0.0,4.0,NaN,1.0,11.0,413.0,181.0,...,93.0,117.0,1.0,NaN,ALABAMA - Metropolitan Counties,NaN,NaN,NaN,NaN,"C, Alabama"


(2777, 56)

In [236]:
# explicitly create columns using the name of the state and county separated by ", "
all_states = crimes_df.State.dropna().tolist()
# the | command allows us to extract the name of all states that follow ", "
state_regex = ", ({})".format("|".join(all_states))
crimes_df.State = crimes_df.Area.str.extract(state_regex)
assert crimes_df.State.isnull().sum() == 0
crimes_df.County = crimes_df.Area.str.replace(state_regex,"")
assert crimes_df.County.isnull().sum() == 0
assert crimes_df.Area.duplicated().sum() == 0

# observe all county names with spaces in their names.
crimes_df.County.loc[crimes_df.County.apply(lambda county: len(county.split()) > 1)]

69         Santa Cruz
80       Little River
128     Saint Francis
131         Van Buren
137      Contra Costa
            ...      
2667         Santa Fe
2688         Van Wert
2700    Oglala Lakota
2704        Van Buren
2708    San Augustine
Name: County, Length: 173, dtype: object

In [237]:
# contains estimates for the population size in each county area name.
pop_est_df = pd.read_csv("county population data/population estimates.csv")
pop_est_df.head(20)

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2015,RDOMESTICMIG2016,RDOMESTICMIG2017,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017
0,40,3,6,1,0,Alabama,Alabama,4779736,4780135,4785579,...,-0.317205,-0.404473,0.788882,0.450741,0.939393,1.364296,0.694271,0.678575,0.558931,1.708218
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54750,...,-1.950739,4.831269,1.047102,5.911832,-6.102101,-4.050282,2.099325,-1.659040,5.103709,1.317904
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183110,...,17.047872,20.493601,22.383175,16.285940,17.196786,22.615285,20.380904,17.903749,21.317244,23.163873
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27332,...,-16.222436,-18.755525,-19.042395,0.256021,-6.822433,-8.018920,-5.549762,-16.411069,-18.947692,-19.159940
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22872,...,0.931388,-1.416117,-0.882983,-5.041980,-4.096646,-5.890038,1.243450,1.818424,-0.531044,0.000000
5,50,3,6,1,9,Alabama,Blount County,57322,57324,57381,...,-1.563369,-1.736835,6.212416,0.243599,-1.354672,-0.486035,-1.771310,-0.538494,-0.659997,7.285313
6,50,3,6,1,11,Alabama,Bullock County,10914,10911,10880,...,-25.222833,-2.492809,-16.385542,-23.010902,-6.388876,-6.993998,10.555084,-23.610848,-0.767018,-14.650602
7,50,3,6,1,13,Alabama,Butler County,20947,20946,20944,...,-7.561530,-7.629781,-6.081930,-3.682096,-7.842571,-14.515343,-0.245706,-7.215578,-7.280706,-5.730083
8,50,3,6,1,15,Alabama,Calhoun County,118572,118586,118466,...,-3.767618,-4.321322,-2.150556,-6.315317,-4.765876,-5.391505,-3.949695,-3.102236,-3.609779,-1.436606
9,50,3,6,1,17,Alabama,Chambers County,34215,34170,34122,...,0.912892,-6.146118,1.927925,-1.437941,4.168930,2.316240,-4.201557,1.413511,-5.643791,2.432152


In [238]:
pop_est_df.isnull().sum()

SUMLEV         0
REGION         0
DIVISION       0
STATE          0
COUNTY         0
              ..
RNETMIG2013    0
RNETMIG2014    0
RNETMIG2015    0
RNETMIG2016    0
RNETMIG2017    0
Length: 132, dtype: int64

In [239]:
# has duplicated information about the District of Columbia that is necessary for this analysis. We will delete
# one of these rows.
pop_est_df.rename(columns={"CTYNAME":"Raw County Data","COUNTY":"CNTY_ID",'STNAME':"State","STATE":"ST_ID"},inplace=True)
print("Total Duplicated County Names:",pop_est_df[['State',"Raw County Data"]].duplicated().sum())
pop_est_df.loc[pop_est_df["Raw County Data"] == "District of Columbia",:]

Total Duplicated County Names: 1


,SUMLEV,REGION,DIVISION,ST_ID,CNTY_ID,State,Raw County Data,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2015,RDOMESTICMIG2016,RDOMESTICMIG2017,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017
327,40,3,5,11,0,District of Columbia,District of Columbia,601723,601766,605040,...,4.364346,4.343174,1.671615,17.028243,16.479745,15.018542,9.015105,10.648405,10.555077,7.708001
328,50,3,5,11,1,District of Columbia,District of Columbia,601723,601766,605040,...,4.364346,4.343174,1.671615,17.028243,16.479745,15.018542,9.015105,10.648405,10.555077,7.708001


In [240]:
pop_est_df=pop_est_df.drop_duplicates(['State',"Raw County Data"]) # drops the duplicated county name

# make standard the manner in which the data is presented in each individual column
pop_est_df=pop_est_df.loc[pop_est_df["Raw County Data"] != pop_est_df['State']].reset_index(drop=True)
pop_est_df['County'] = pop_est_df['Raw County Data'].apply(rm_all_denotations).str.replace("St\.","Saint")
pop_est_df['Area'] = pop_est_df['County'] + ', ' + pop_est_df['State']
pop_est_df['Loc Abbrev'] = pop_est_df['County'].str[0].str.upper() + ', ' + pop_est_df["State"]
pop_est_df.head()

,SUMLEV,REGION,DIVISION,ST_ID,CNTY_ID,State,Raw County Data,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,County,Area,Loc Abbrev
0,50,3,6,1,1,Alabama,Autauga County,54571,54571,54750,...,5.911832,-6.102101,-4.050282,2.099325,-1.659040,5.103709,1.317904,Autauga,"Autauga, Alabama","A, Alabama"
1,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183110,...,16.285940,17.196786,22.615285,20.380904,17.903749,21.317244,23.163873,Baldwin,"Baldwin, Alabama","B, Alabama"
2,50,3,6,1,5,Alabama,Barbour County,27457,27457,27332,...,0.256021,-6.822433,-8.018920,-5.549762,-16.411069,-18.947692,-19.159940,Barbour,"Barbour, Alabama","B, Alabama"
3,50,3,6,1,7,Alabama,Bibb County,22915,22919,22872,...,-5.041980,-4.096646,-5.890038,1.243450,1.818424,-0.531044,0.000000,Bibb,"Bibb, Alabama","B, Alabama"
4,50,3,6,1,9,Alabama,Blount County,57322,57324,57381,...,0.243599,-1.354672,-0.486035,-1.771310,-0.538494,-0.659997,7.285313,Blount,"Blount, Alabama","B, Alabama"


In [241]:
pop_est_df.dtypes

SUMLEV           int64
REGION           int64
DIVISION         int64
ST_ID            int64
CNTY_ID          int64
                ...   
RNETMIG2016    float64
RNETMIG2017    float64
County          object
Area            object
Loc Abbrev      object
Length: 135, dtype: object

In [242]:
pop_est_df.shape

(3141, 135)

In [243]:
assert crimes_df.County.str.contains("St\.|[cC]ity| Borough|[cC]ensus|[aA]rea").sum() == 0

In [244]:
states = pop_est_df.State.unique()
#def rm_non_counties():

# the following function can be used to determine whether a given area has only "Area", "City", or "Borough"
# reported once and not data and not "County" data.Areas without "Area", "City", or "Borough" in their names are
#  assumed to report county-level data.
def find_states_without_counties(state,county_data=pop_est_df):
    state_subset = county_data.loc[county_data.State == state].copy()
    #~pop_est_df['Raw County Data'].loc[pop_est_df['Raw County Data'].str.contains("County")]
    county_substr = state_subset["Raw County Data"].apply(rm_all_denotations)
    #dedicated counties:
    #state_counties = state_subset.loc[state_subset['Raw County Data'].str.contains("[cC]ounty"),'Raw County Data'].values
    freq=county_substr.value_counts()#.loc[~freq.index.isin(state_counties)]
    unique_place_names = freq.loc[(freq==1)].index # & (~freq.index.isin(state_counties))].index.tolist()
    return state_subset["Raw County Data"].loc[
        (state_subset["Raw County Data"].str.contains("|".join(unique_place_names))) &
        (state_subset["Raw County Data"].str.contains('[Cc]ity|[bB]orough.*|Census Area.*')) &
        (~state_subset["Raw County Data"].str.contains("[cC]ounty"))
    ]
    
unique_denotations = {state:find_states_without_counties(state) for state in states}
unique_denotations = {state:only_non_counties.tolist() for state, only_non_counties in unique_denotations.items() if not only_non_counties.empty}
unique_denotations # prints out all areas that do not appear to be counties
#pop_est_df.groupby("State")['Raw County Data'].apply(lambda x: x.value_counts())

{'Alaska': ['Aleutians East Borough',
  'Aleutians West Census Area',
  'Bethel Census Area',
  'Bristol Bay Borough',
  'Denali Borough',
  'Dillingham Census Area',
  'Fairbanks North Star Borough',
  'Haines Borough',
  'Hoonah-Angoon Census Area',
  'Juneau City and Borough',
  'Kenai Peninsula Borough',
  'Ketchikan Gateway Borough',
  'Kodiak Island Borough',
  'Kusilvak Census Area',
  'Lake and Peninsula Borough',
  'Matanuska-Susitna Borough',
  'Nome Census Area',
  'North Slope Borough',
  'Northwest Arctic Borough',
  'Petersburg Borough',
  'Prince of Wales-Hyder Census Area',
  'Sitka City and Borough',
  'Southeast Fairbanks Census Area',
  'Valdez-Cordova Census Area',
  'Wrangell City and Borough',
  'Yakutat City and Borough',
  'Yukon-Koyukuk Census Area'],
 'Nevada': ['Carson City'],
 'Virginia': ['Alexandria city',
  'Bristol city',
  'Buena Vista city',
  'Charlottesville city',
  'Chesapeake city',
  'Colonial Heights city',
  'Covington city',
  'Danville city',

In [245]:
# of all areas that do not appear to by counties in the population estimate data set, 
# only Carson city could be found in the crimes data set.
for state, areas in unique_denotations.items():
    for area in areas:
        place = rm_all_denotations(area).title() + ".*, " + state
        queried_area = crimes_df.loc[crimes_df.Area.str.contains(place)]
        if not queried_area.empty:
            print(area+ ', ' + state + ' was found in crimes dataset:')
            display(queried_area)
            print("\n\n")

Carson City, Nevada was found in crimes dataset:


,State,Raw County Data (2015),Violent crime (2015),Murder and nonnegligent manslaughter (2015),Rape (revised) (2015),Rape (previous def) (2015),Robbery (2015),Aggravated assault (2015),Property crime (2015),Burglary (2015),...,Burglary (2017),Larceny-theft (2017),Motor vehicle theft (2017),Arson (2017),State/County Type (2017),notice num 1 (2017),notice num 2 (2017),notice 1 (2017),notice 2 (2017),Loc Abbrev
1402,Nevada,Carson City,172.0,1.0,2.0,NaN,20.0,149.0,1000.0,184.0,...,166.0,632.0,81.0,7.0,NEVADA - Metropolitan Counties,NaN,NaN,NaN,NaN,"C, Nevada"


In [246]:
duped_areas = pop_est_df.Area.loc[pop_est_df.Area.duplicated()].values
print("duplicated population estimation area names:")
duped_areas

duplicated population estimation area names:


array(['Baltimore, Maryland', 'Saint Louis, Missouri',
       'Fairfax, Virginia', 'Franklin, Virginia', 'Richmond, Virginia',
       'Roanoke, Virginia'], dtype=object)

In [247]:
pop_est_df.loc[(pop_est_df.Area.isin(duped_areas))]

,SUMLEV,REGION,DIVISION,ST_ID,CNTY_ID,State,Raw County Data,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,County,Area,Loc Abbrev
1194,50,3,5,24,5,Maryland,Baltimore County,805029,805150,806405,...,5.727040,4.026073,3.909992,1.299466,1.165512,0.721335,-0.385841,Baltimore,"Baltimore, Maryland","B, Maryland"
1215,50,3,5,24,510,Maryland,Baltimore city,620961,620951,621026,...,-4.575377,-0.504731,-4.291443,-2.725796,-5.813520,-12.774307,-11.276194,Baltimore,"Baltimore, Maryland","B, Maryland"
1577,50,2,4,29,189,Missouri,St. Louis County,998954,998882,998681,...,-1.905916,-1.232117,-1.603069,-2.025353,-1.549378,-4.487474,-2.988558,Saint Louis,"Saint Louis, Missouri","S, Missouri"
1596,50,2,4,29,510,Missouri,St. Louis city,319294,319371,319365,...,-6.230549,-5.328405,-7.514837,-8.483326,-9.552216,-14.550342,-18.624250,Saint Louis,"Saint Louis, Missouri","S, Missouri"
2847,50,3,5,51,59,Virginia,Fairfax County,1081726,1081682,1086734,...,7.667208,5.057565,2.932715,-6.009446,-6.352869,-5.670725,-3.971097,Fairfax,"Fairfax, Virginia","F, Virginia"
2851,50,3,5,51,67,Virginia,Franklin County,56159,56139,56189,...,3.607863,0.106532,0.195420,1.776972,1.564403,-0.249132,7.246634,Franklin,"Franklin, Virginia","F, Virginia"
2894,50,3,5,51,159,Virginia,Richmond County,9254,9254,9279,...,-4.435789,-8.420362,-10.976827,-2.801748,-3.053435,0.910177,20.086893,Richmond,"Richmond, Virginia","R, Virginia"
2895,50,3,5,51,161,Virginia,Roanoke County,92376,92474,92414,...,6.390984,0.646576,7.587971,1.135475,2.228603,2.721540,6.403233,Roanoke,"Roanoke, Virginia","R, Virginia"
2923,50,3,5,51,600,Virginia,Fairfax city,22565,22542,22612,...,-7.495454,11.416230,12.131190,-3.607163,-4.453199,-1.778493,-5.800851,Fairfax,"Fairfax, Virginia","F, Virginia"
2925,50,3,5,51,620,Virginia,Franklin city,8582,8580,8581,...,-11.627224,3.425467,7.518355,-16.414738,-1.913418,-9.421997,-3.169959,Franklin,"Franklin, Virginia","F, Virginia"


In [248]:
# the following gets all areas that are either not duplicated or do not contain the word "city" in them.
pop_est_df = pop_est_df.loc[(~pop_est_df.Area.isin(duped_areas)) | (~pop_est_df['Raw County Data'].str.contains("[cC]ity"))]

In [249]:
# the following code is used to determine whether all counties are similarly spelled across years
# that do not contain null values for the "Raw County Data" column.
raw_county_data_cols = crimes_df.columns[crimes_df.columns.str.contains('Raw County Data')].tolist()
# replaces "Police Department, numbers, and 'County' with an empty string
all_raw_counties=crimes_df[raw_county_data_cols].loc[crimes_df[raw_county_data_cols].notnull().all(axis=1)].apply(lambda col:col.str.replace(" *\d+.*| Police Dep.*| County","").str.strip())

# determines whether all merged county data (without any null values from 2015-2017) are equally named within
# the raw county names column.
msk=((all_raw_counties[raw_county_data_cols[0]]==all_raw_counties[raw_county_data_cols[1]]) &
(all_raw_counties[raw_county_data_cols[1]]==all_raw_counties[raw_county_data_cols[1]]) &
(all_raw_counties[raw_county_data_cols[0]]==all_raw_counties[raw_county_data_cols[2]]))

print("Values in agreement: {:.2%}".format(msk.sum()/len(msk)))
all_raw_counties.loc[~msk]
# The Raw County Data From the Initial Files were left unmodifed:

Values in agreement: 100.00%


,Raw County Data (2015),Raw County Data (2016),Raw County Data (2017)


In [250]:
# it was already determined that where no null values were found, that the raw county names were similar for each
# row across years 2015-2017. The following is another way of verifying such.
consistent_indices=all_raw_counties.index
for rc1,rc2 in combinations(raw_county_data_cols,2):
    pair_raw_counties=crimes_df[raw_county_data_cols].loc[(crimes_df[raw_county_data_cols].notnull().all(axis=1)) & ~(crimes_df.index.isin(consistent_indices))].apply(lambda col:col.str.replace(" *\d+.*","").str.strip())
    print((pair_raw_counties[rc1] == pair_raw_counties[rc2]).all())

True
True
True


In [251]:
def join_raw_county_data(raw_county_row):
    if raw_county_row.notnull().all():
        return re.sub(" *\d+.*| Police Dep.*| County","",raw_county_row.tolist()[0])
    #elif:
        #raw_county_row.isnull().sum() == 1:
        #return re.sub(" *\d+.*","",raw_county_row.loc[pd.notnull(raw_county_row)].values[0]).strip()
    else:
        return re.sub(" *\d+.*","",raw_county_row.loc[pd.notnull(raw_county_row)].values[0]).strip()
    
# removes "Police Dep...., County, and numbers from each county name and makes it a separate column"
crimes_df['Raw County Data'] = crimes_df[raw_county_data_cols].apply(join_raw_county_data,axis=1)

In [252]:
assert crimes_df['Raw County Data'].isnull().sum() == 0

In [253]:
# removes all areas that contain's the word "City" from the crimes data set and population estimates data set
crimes_df = crimes_df.loc[~crimes_df['Raw County Data'].str.contains("[cC]ity")]
pop_est_df= pop_est_df.loc[~(pop_est_df['Raw County Data'].str.contains('[cC]ity'))]#|
                           #(pop_est_df['Raw County Data'].str.contains('[cC]ounty'))]


In [254]:
find_unique_locations(crimes_df,pop_est_df)

['Augusta-Richmond, Georgia',
 'De Kalb, Illinois',
 'De Kalb, Indiana',
 'Dona Ana, New Mexico',
 'Hartsville/Trousdale, Tennessee',
 'La Porte, Indiana',
 'La Salle, Illinois',
 'La Salle, Louisiana',
 'Lac Qui Parle, Minnesota',
 'Lamoure, North Dakota',
 'Shannon, South Dakota',
 'Story, Nevada',
 'Westchester Public Safety, New York']

In [255]:
pop_est_df.loc[(pop_est_df['Raw County Data'].str.contains('[cC]ity'))]

,SUMLEV,REGION,DIVISION,ST_ID,CNTY_ID,State,Raw County Data,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,County,Area,Loc Abbrev


In [256]:
#crimes_df.loc[crimes_df['Area'].str.contains("Charles|James*")]

In [257]:
# is used to search for area names in the chosen data set (default population estimates data set)
def search_for(place,dataset=pop_est_df):
    data_columns = ["SEARCH"]+dataset.columns.tolist()
    query = dataset.loc[dataset['Area'].str.contains(place).fillna(False)].copy()
    return query.assign(SEARCH=place)[data_columns]# if not query.empty else query

In [258]:
#columns not found:
display(search_for('Sh.*n.*n.*Dakota'))

,SEARCH,SUMLEV,REGION,DIVISION,ST_ID,CNTY_ID,State,Raw County Data,CENSUS2010POP,ESTIMATESBASE2010,...,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,County,Area,Loc Abbrev


In [259]:
# finding occurrences of the counties Shannon, South Dakota and Story, Nevada
comparison_counties = crimes_df.loc[crimes_df.Area.str.contains("Shannon.*South Dakota|Story.*, Nevada"),['State','Area',"County","Loc Abbrev"]].copy()
comparison_counties["Loc Abbrev"] = comparison_counties["County"].apply(lambda county_name:county_name[0]) + ", " + comparison_counties["State"]
pop_est_comparisons = pop_est_df.loc[pop_est_df['Loc Abbrev'].isin(comparison_counties['Loc Abbrev'])]
pop_est_comparisons

,SUMLEV,REGION,DIVISION,ST_ID,CNTY_ID,State,Raw County Data,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,County,Area,Loc Abbrev
1759,50,4,8,32,29,Nevada,Storey County,4010,4010,3994,...,-20.263425,-14.734393,-5.508197,4.217185,20.078227,30.530467,3.752345,Storey,"Storey, Nevada","S, Nevada"
2416,50,2,4,46,111,South Dakota,Sanborn County,2355,2355,2354,...,-0.849437,-13.267708,2.153316,1.288660,1.711230,7.198814,21.541011,Sanborn,"Sanborn, South Dakota","S, South Dakota"
2417,50,2,4,46,115,South Dakota,Spink County,6415,6415,6421,...,11.923196,20.562029,-10.302250,-3.963113,-14.139706,-7.930954,0.624317,Spink,"Spink, South Dakota","S, South Dakota"
2418,50,2,4,46,117,South Dakota,Stanley County,2966,2966,2971,...,-3.698101,-8.061807,-3.022670,-5.034402,-7.388749,3.333889,-12.924606,Stanley,"Stanley, South Dakota","S, South Dakota"
2419,50,2,4,46,119,South Dakota,Sully County,1373,1368,1378,...,-2.903811,29.935852,8.368201,-28.792135,-7.104796,4.955752,-16.277424,Sully,"Sully, South Dakota","S, South Dakota"


In [260]:
# not_found_idx=crimes_df.loc[crimes_df.Area.str.contains('Shannon.*South Dakota')].index
# crimes_df=crimes_df.drop(not_found_idx)

In [261]:
crimes_unique_ct=find_unique_locations(crimes_df,pop_est_df)
# not in population estimates df:
crimes_unique_ct

['Augusta-Richmond, Georgia',
 'De Kalb, Illinois',
 'De Kalb, Indiana',
 'Dona Ana, New Mexico',
 'Hartsville/Trousdale, Tennessee',
 'La Porte, Indiana',
 'La Salle, Illinois',
 'La Salle, Louisiana',
 'Lac Qui Parle, Minnesota',
 'Lamoure, North Dakota',
 'Shannon, South Dakota',
 'Story, Nevada',
 'Westchester Public Safety, New York']

In [262]:
# this is how values in the crimes data set will be regex searched for the purpose of renaming
alt_area_names_dict = {
 'Augusta-Richmond, Georgia': 'Augusta.*, Georgia|Richmond.*, Georgia',
 'De Kalb, Illinois': '[De].[kK]alb.*, Illinois',
 'De Kalb, Indiana': '[De].[kK]alb.*, Indiana',
 'Dona Ana, New Mexico': 'Ana.*New Mexico',
 'Hartsville/Trousdale, Tennessee': 'Hartsville|Trousdale',
 'La Porte, Indiana': 'La.*Porte.*, Indiana',
 'La Salle, Illinois': 'La.*[sS]alle.*, Illinois',
 'La Salle, Louisiana': 'La.*[sS]alle.*, Louisiana',
 'Lac Qui Parle, Minnesota': 'Parle.*, Minn',
 'Lamoure, North Dakota': 'La.*[mM]oure.*, North Dakota',
 'Story, Nevada': 'Stor.*y.*, Nevada',
 'Westchester Public Safety, New York': 'Westchester.*, New York'
}
#def modify names:
alt_area_names_dict

{'Augusta-Richmond, Georgia': 'Augusta.*, Georgia|Richmond.*, Georgia',
 'De Kalb, Illinois': '[De].[kK]alb.*, Illinois',
 'De Kalb, Indiana': '[De].[kK]alb.*, Indiana',
 'Dona Ana, New Mexico': 'Ana.*New Mexico',
 'Hartsville/Trousdale, Tennessee': 'Hartsville|Trousdale',
 'La Porte, Indiana': 'La.*Porte.*, Indiana',
 'La Salle, Illinois': 'La.*[sS]alle.*, Illinois',
 'La Salle, Louisiana': 'La.*[sS]alle.*, Louisiana',
 'Lac Qui Parle, Minnesota': 'Parle.*, Minn',
 'Lamoure, North Dakota': 'La.*[mM]oure.*, North Dakota',
 'Story, Nevada': 'Stor.*y.*, Nevada',
 'Westchester Public Safety, New York': 'Westchester.*, New York'}

In [263]:
alt_area_names=np.array(list(alt_area_names_dict.values()))
terms_dataset = pd.concat(list(map(search_for,alt_area_names)))
included_terms = np.isin(alt_area_names,terms_dataset.SEARCH)
print("All search terms matched:",included_terms.all())
print("Missing:",alt_area_names[~included_terms])
terms_dataset

All search terms matched: True
Missing: []


,SEARCH,SUMLEV,REGION,DIVISION,ST_ID,CNTY_ID,State,Raw County Data,CENSUS2010POP,ESTIMATESBASE2010,...,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,County,Area,Loc Abbrev
506,"Augusta.*, Georgia|Richmond.*, Georgia",50,3,5,13,245,Georgia,Richmond County,200549,200549,...,-8.169308,0.288581,-7.896089,-4.109991,-3.623827,-1.952404,-5.611578,Richmond,"Richmond, Georgia","R, Georgia"
612,"[De].[kK]alb.*, Illinois",50,2,3,17,37,Illinois,DeKalb County,105160,105160,...,-13.065658,-5.950499,-6.428678,0.325706,-7.156012,-4.424143,-1.787934,DeKalb,"DeKalb, Illinois","D, Illinois"
712,"[De].[kK]alb.*, Indiana",50,2,3,18,33,Indiana,DeKalb County,42223,42219,...,0.684390,-7.647461,-1.229184,-1.038814,-0.942963,0.752304,2.387193,DeKalb,"DeKalb, Indiana","D, Indiana"
1801,Ana.*New Mexico,50,4,8,35,13,New Mexico,Do�a Ana County,209233,209222,...,3.858393,-2.084749,-10.620504,-7.739818,-7.003389,-2.969284,1.816264,Do�a Ana,"Do�a Ana, New Mexico","D, New Mexico"
2511,Hartsville|Trousdale,50,3,6,47,169,Tennessee,Trousdale County,7870,7864,...,-7.778628,-3.077515,2.950988,22.540205,5.239194,210.894942,11.081715,Trousdale,"Trousdale, Tennessee","T, Tennessee"
741,"La.*Porte.*, Indiana",50,2,3,18,91,Indiana,LaPorte County,111467,111467,...,-2.908869,-1.383654,-0.233494,1.021821,-8.564675,-6.903946,-2.787906,LaPorte,"LaPorte, Indiana","L, Indiana"
643,"La.*[sS]alle.*, Illinois",50,2,3,17,99,Illinois,LaSalle County,113924,113915,...,-2.093974,-3.957387,-2.618370,-6.122103,-4.318568,-4.973283,-4.343214,LaSalle,"LaSalle, Illinois","L, Illinois"
1141,"La.*[sS]alle.*, Louisiana",50,3,7,22,59,Louisiana,LaSalle Parish,14890,14890,...,0.871401,-4.431910,-2.089935,2.829998,5.694379,1.997603,-8.204102,LaSalle,"LaSalle, Louisiana","L, Louisiana"
1349,"Parle.*, Minn",50,2,4,27,73,Minnesota,Lac qui Parle County,7259,7259,...,0.000000,-8.368784,-13.594845,-11.796864,-2.324061,-16.161023,-3.723563,Lac qui Parle,"Lac qui Parle, Minnesota","L, Minnesota"
2011,"La.*[mM]oure.*, North Dakota",50,2,4,38,45,North Dakota,LaMoure County,4139,4139,...,-3.892471,0.000000,2.199951,0.244111,6.562956,-5.577110,-6.584563,LaMoure,"LaMoure, North Dakota","L, North Dakota"


In [264]:
def modify_alternate_names(area_data,alt_area_names_dict=alt_area_names_dict,search_dataset=pop_est_df):
    initial_area_name = area_data.Area
    alt_name_regex = alt_area_names_dict.get(initial_area_name)
    if alt_name_regex is not None:
        query = search_for(alt_name_regex,dataset=search_dataset)
        assert query.shape[0] <= 1, display("Multiple similar areas queried:",query)
        if not query.empty:
            area_data.Area = query.Area.values[0]
            area_data.County = query.County.values[0]
            print(initial_area_name,"modified to", area_data.Area,'\n\n')
        else:
            print(initial_area_name, "returned no results\nsearch:",alt_name_regex,'\n\n')
    return area_data
crimes_df = crimes_df.apply(modify_alternate_names,search_dataset = pop_est_df, axis=1)

La Salle, Illinois modified to LaSalle, Illinois 


La Porte, Indiana modified to LaPorte, Indiana 


De Kalb, Indiana modified to DeKalb, Indiana 


La Salle, Louisiana modified to LaSalle, Louisiana 


Lac Qui Parle, Minnesota modified to Lac qui Parle, Minnesota 


Story, Nevada modified to Storey, Nevada 


Dona Ana, New Mexico modified to Do�a Ana, New Mexico 


Westchester Public Safety, New York modified to Westchester, New York 


Lamoure, North Dakota modified to LaMoure, North Dakota 


Hartsville/Trousdale, Tennessee modified to Trousdale, Tennessee 


Augusta-Richmond, Georgia modified to Richmond, Georgia 


De Kalb, Illinois modified to DeKalb, Illinois 




In [265]:
# assert that there is a completely 1 to one relationship between all counties in both datasets
# and that all counties aside from Shannon Nevada within the crimes_df dataset are found within the population estimate datasets
# (not vice versa)
#crimes_unique_ct = sorted(set(crime_ct).difference(est_ct.values))

excluding_shannon=(~crimes_df.Area.str.contains("Shannon.*, South Dakota")).values
assert not find_unique_locations(crimes_df.loc[excluding_shannon],pop_est_df) and crimes_df.Area.duplicated().sum() == 0 and pop_est_df.Area.duplicated().sum() == 0

In [266]:
relevant_columns = ['Area'] + [col for col in pop_est_df.columns if 'POPESTIMATE' in col]
merged_dataset = pd.merge(crimes_df,pop_est_df[relevant_columns],left_on='Area',right_on='Area',how='left')

In [267]:
print("total current rows:")
print(merged_dataset.shape[0])

print("total current states:")
print(merged_dataset.State.unique().shape[0])


total current rows:
2774
total current states:
46


In [268]:
fmr_path = "/home/sbr0/Dropbox/Analysis Planning/coursera stuffs/Capstone/First-Class Restaurant Location Analysis/data sets/fair-market-rents{year}.csv"
fmr_dict = {}
years = [2015,2016,2017]

for year in years:
    current_path = fmr_path.format(year=str(year))
    fmr_dict[year] = pd.read_csv(current_path)
    display(fmr_dict[year].head())    

,fips2000,fips2010,fmr2,fmr0,fmr1,fmr3,fmr4,county,State,CouSub,countyname,Metro_code,Areaname,county_town_name,pop2010,state_alpha,fmr_type,metro
0,100199999.0,100199999,788,628,663,1084,1288,1.0,1,99999,Autauga County,METRO33860M33860,"Montgomery, AL MSA",Autauga County,54571.0,AL,40,1
1,100399999.0,100399999,762,494,643,1123,1318,3.0,1,99999,Baldwin County,NCNTY01003N01003,"Baldwin County, AL",Baldwin County,182265.0,AL,40,0
2,100599999.0,100599999,670,492,495,834,895,5.0,1,99999,Barbour County,NCNTY01005N01005,"Barbour County, AL",Barbour County,27457.0,AL,40,0
3,100799999.0,100799999,773,545,652,1015,1142,7.0,1,99999,Bibb County,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",Bibb County,22915.0,AL,40,1
4,100999999.0,100999999,773,545,652,1015,1142,9.0,1,99999,Blount County,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",Blount County,57322.0,AL,40,1


,fips2000,fips2010,fmr2,fmr0,fmr1,fmr3,fmr4,State,Metro_code,areaname,county,CouSub,countyname,county_town_name,pop2010,state_alpha,fmr_type,metro
0,100199999.0,100199999,788,565,647,1029,1336,1,METRO33860M33860,"Montgomery, AL MSA",1.0,99999,Autauga County,Autauga County,54571.0,AL,40,1
1,100399999.0,100399999,873,719,754,1272,1472,1,METRO19300M19300,"Daphne-Fairhope-Foley, AL MSA",3.0,99999,Baldwin County,Baldwin County,182265.0,AL,40,1
2,100599999.0,100599999,636,472,475,789,872,1,NCNTY01005N01005,"Barbour County, AL",5.0,99999,Barbour County,Barbour County,27457.0,AL,40,0
3,100799999.0,100799999,840,613,726,1134,1250,1,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",7.0,99999,Bibb County,Bibb County,22915.0,AL,40,1
4,100999999.0,100999999,840,613,726,1134,1250,1,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",9.0,99999,Blount County,Blount County,57322.0,AL,40,1


,fips2000,fips2010,fmr2,fmr0,fmr1,fmr3,fmr4,State,Metro_code,areaname,county,CouSub,countyname,county_town_name,pop2010,state_alpha,fmr_type,metro
0,100199999.0,100199999,788,565,647,1029,1336,1,METRO33860M33860,"Montgomery, AL MSA",1.0,99999,Autauga County,Autauga County,54571.0,AL,40,1
1,100399999.0,100399999,873,719,754,1272,1472,1,METRO19300M19300,"Daphne-Fairhope-Foley, AL MSA",3.0,99999,Baldwin County,Baldwin County,182265.0,AL,40,1
2,100599999.0,100599999,636,472,475,789,872,1,NCNTY01005N01005,"Barbour County, AL",5.0,99999,Barbour County,Barbour County,27457.0,AL,40,0
3,100799999.0,100799999,840,613,726,1134,1250,1,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",7.0,99999,Bibb County,Bibb County,22915.0,AL,40,1
4,100999999.0,100999999,840,613,726,1134,1250,1,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",9.0,99999,Blount County,Blount County,57322.0,AL,40,1


In [269]:
def strip_whitespaces(variable):
    try:
        return variable.str.strip()
    except AttributeError:
        return variable
    
for year in years:
    current_path = fmr_path.format(year=str(year))
    fmr_dict[year] = pd.read_csv(current_path)
    display(fmr_dict[year].head())
    if "Areaname" in fmr_dict[year]:
        fmr_dict[year].rename(columns={"Areaname":"areaname"},inplace=True)
    fmr_dict[year]=fmr_dict[year].apply(strip_whitespaces)

,fips2000,fips2010,fmr2,fmr0,fmr1,fmr3,fmr4,county,State,CouSub,countyname,Metro_code,Areaname,county_town_name,pop2010,state_alpha,fmr_type,metro
0,100199999.0,100199999,788,628,663,1084,1288,1.0,1,99999,Autauga County,METRO33860M33860,"Montgomery, AL MSA",Autauga County,54571.0,AL,40,1
1,100399999.0,100399999,762,494,643,1123,1318,3.0,1,99999,Baldwin County,NCNTY01003N01003,"Baldwin County, AL",Baldwin County,182265.0,AL,40,0
2,100599999.0,100599999,670,492,495,834,895,5.0,1,99999,Barbour County,NCNTY01005N01005,"Barbour County, AL",Barbour County,27457.0,AL,40,0
3,100799999.0,100799999,773,545,652,1015,1142,7.0,1,99999,Bibb County,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",Bibb County,22915.0,AL,40,1
4,100999999.0,100999999,773,545,652,1015,1142,9.0,1,99999,Blount County,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",Blount County,57322.0,AL,40,1


,fips2000,fips2010,fmr2,fmr0,fmr1,fmr3,fmr4,State,Metro_code,areaname,county,CouSub,countyname,county_town_name,pop2010,state_alpha,fmr_type,metro
0,100199999.0,100199999,788,565,647,1029,1336,1,METRO33860M33860,"Montgomery, AL MSA",1.0,99999,Autauga County,Autauga County,54571.0,AL,40,1
1,100399999.0,100399999,873,719,754,1272,1472,1,METRO19300M19300,"Daphne-Fairhope-Foley, AL MSA",3.0,99999,Baldwin County,Baldwin County,182265.0,AL,40,1
2,100599999.0,100599999,636,472,475,789,872,1,NCNTY01005N01005,"Barbour County, AL",5.0,99999,Barbour County,Barbour County,27457.0,AL,40,0
3,100799999.0,100799999,840,613,726,1134,1250,1,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",7.0,99999,Bibb County,Bibb County,22915.0,AL,40,1
4,100999999.0,100999999,840,613,726,1134,1250,1,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",9.0,99999,Blount County,Blount County,57322.0,AL,40,1


,fips2000,fips2010,fmr2,fmr0,fmr1,fmr3,fmr4,State,Metro_code,areaname,county,CouSub,countyname,county_town_name,pop2010,state_alpha,fmr_type,metro
0,100199999.0,100199999,788,565,647,1029,1336,1,METRO33860M33860,"Montgomery, AL MSA",1.0,99999,Autauga County,Autauga County,54571.0,AL,40,1
1,100399999.0,100399999,873,719,754,1272,1472,1,METRO19300M19300,"Daphne-Fairhope-Foley, AL MSA",3.0,99999,Baldwin County,Baldwin County,182265.0,AL,40,1
2,100599999.0,100599999,636,472,475,789,872,1,NCNTY01005N01005,"Barbour County, AL",5.0,99999,Barbour County,Barbour County,27457.0,AL,40,0
3,100799999.0,100799999,840,613,726,1134,1250,1,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",7.0,99999,Bibb County,Bibb County,22915.0,AL,40,1
4,100999999.0,100999999,840,613,726,1134,1250,1,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",9.0,99999,Blount County,Blount County,57322.0,AL,40,1


In [270]:
# rename fmr columns and remove references to cities, boroughs, etc.
for year in fmr_dict:
    fmr_dict[year] = fmr_dict[year].rename(columns={"county":"C_ID","countyname":"Raw County Data",area_name:"Raw Area Name"})
    fmr_dict[year]['County']=fmr_dict[year]['Raw County Data'].str.replace("^St\.",'Saint').apply(rm_all_denotations)

In [271]:
# get all abbreviations for states
# assert that all states in the FMR rent data and merged data set are contained in the abbreviations list
state_abbreviations= pd.read_csv("/home/sbr0/Dropbox/Analysis Planning/coursera stuffs/Capstone/First-Class Restaurant Location Analysis/data sets/states.csv")
assert merged_dataset.State.loc[~(merged_dataset.State.isin(state_abbreviations.State))].empty
for year in fmr_dict:
    assert state_abbreviations.loc[~(state_abbreviations.Abbreviation.isin(fmr_dict[year].state_alpha))].empty


In [272]:
# map abbreviations in the fmr data set to the fully spelled out states
# and drop null rows
for year in fmr_dict:
    abbrev_dict = pd.Series(data =state_abbreviations.State.values,
                            index = state_abbreviations.Abbreviation.values).to_dict()
    fmr_dict[year]['State'] = fmr_dict[year].state_alpha.map(abbrev_dict)
    null_idx = fmr_dict[year].loc[fmr_dict[year]['State'].isnull()].index
    fmr_dict[year]=fmr_dict[year].drop(null_idx)
    assert merged_dataset['State'].isin(fmr_dict[year]['State']).all()

In [273]:
# make standard columns that can be found in the merged data set:
for year in fmr_dict:
    fmr_dict[year]['Area'] = fmr_dict[year]['County'] + ", " + fmr_dict[year]['State']
    fmr_dict[year]['Loc Abbrev'] = fmr_dict[year]['County'].str[0].str.upper() + ', '+ fmr_dict[year]['State']
    fmr_dict[year][['Area',"Loc Abbrev"]]

In [274]:

from itertools import combinations
def get_colnames(regex,dataset):
    columns=dataset.columns[dataset.columns.astype(str).str.contains(regex)].values.tolist()
    return columns[0] if len(columns) == 1 else columns

area_names_dict={}
print("Free Market Counties: 2015-2017")
for year,col in fmr_dict.items():
    print("total areas for {}:".format(year),col.shape[0])
    area_names_dict[year] = col["Area"].unique()

year_combinations=combinations(fmr_dict.keys(),2)

# the years 2016 and 2017 contained no differing column sets
combinations_w_missing = [(y1,y2) for y1,y2 in year_combinations if set(fmr_dict[y1]["Area"]).symmetric_difference(fmr_dict[y2]["Area"])]
for x,y in combinations_w_missing:
    #print(x,str(y)+':')
    var = get_colnames("[aA]reaname",col)
    missing_counties = set(fmr_dict[x]["Area"]).symmetric_difference(fmr_dict[y]["Area"])
    missing = len(missing_counties)
    print()
    print("Years:",x,'and',y,'\nTotal Missing:',missing)
    print("Missing Counties:",missing_counties)
    print()
    

Free Market Counties: 2015-2017
total areas for 2015: 4687
total areas for 2016: 4686
total areas for 2017: 4686

Years: 2015 and 2016 
Total Missing: 1
Missing Counties: {'Columbia, Maryland'}


Years: 2015 and 2017 
Total Missing: 1
Missing Counties: {'Columbia, Maryland'}



In [275]:
fmr_dict[2016].loc[(fmr_dict[2016]['Area'].str.contains("C.*l.*mb.*")) & (fmr_dict[2016]['State'].str.contains("Maryland"))]

,fips2000,fips2010,fmr2,fmr0,fmr1,fmr3,fmr4,State,Metro_code,areaname,...,CouSub,Raw County Data,county_town_name,pop2010,state_alpha,fmr_type,metro,County,Area,Loc Abbrev


In [276]:
fmr_dict[2017].loc[(fmr_dict[2017]['Area'].str.contains("C.*l.*mb.*")) & (fmr_dict[2017]['State'].str.contains("Maryland"))]

,fips2000,fips2010,fmr2,fmr0,fmr1,fmr3,fmr4,State,Metro_code,areaname,...,CouSub,Raw County Data,county_town_name,pop2010,state_alpha,fmr_type,metro,County,Area,Loc Abbrev


In [277]:
print("Total cities in each dataset:")
for year in fmr_dict:
    cities_idx=fmr_dict[year].loc[fmr_dict[year]['Raw County Data'].str.contains('[cC]ity')].index
    print(year,len(cities_idx))

Total cities in each dataset:
2015 51
2016 49
2017 49


In [278]:
# aggregating by county, state while getting the mean price of a house in each given area 
# with 0, 1, 2, 3, and 4 rooms

agg_fmr_dict = {}
for year in fmr_dict:
    agg_fmr_dict[year] = fmr_dict[year].groupby(['State','Area',"County"],as_index=False)[['fmr0','fmr1','fmr2','fmr3','fmr4']].mean()
    print("FMR dataset size for Year,",year)
    print("Before Aggregation:",len(fmr_dict[year]))
    print("After:",len(agg_fmr_dict[year]))

FMR dataset size for Year, 2015
Before Aggregation: 4687
After: 3139
FMR dataset size for Year, 2016
Before Aggregation: 4686
After: 3138
FMR dataset size for Year, 2017
Before Aggregation: 4686
After: 3138


In [279]:
# determines whether any area is still duplicated after aggregating by area name and mean fmr
for year in agg_fmr_dict:
    assert agg_fmr_dict[year].Area.duplicated().sum() == 0


In [280]:
# prepare for merging
for year in [2015,2016,2017]:
    agg_fmr_dict[year]['year'] = year

In [281]:
fmr_cols = get_colnames("fmr",agg_fmr_dict[2015])
static_col_names = ["State","County","Area"] # columns that will remain the same when merging free market data
relevant_columns = fmr_cols + static_col_names # columns to keep

# get the county name and state name for each area, and add 2015 - 2017 to fmr data:
fmr_data_list = [format_data_for_merging(agg_fmr_dict[year][relevant_columns],year,is_column_host= year == 2015,
                                        static_col_names=static_col_names) for year in [2015,2016,2017]]

agg_fmr_df = reduce(counties_merge,fmr_data_list) # performs an outer join on area name
agg_fmr_df

,fmr0 (2015),fmr1 (2015),fmr2 (2015),fmr3 (2015),fmr4 (2015),State,County,Area,fmr0 (2016),fmr1 (2016),fmr2 (2016),fmr3 (2016),fmr4 (2016),fmr0 (2017),fmr1 (2017),fmr2 (2017),fmr3 (2017),fmr4 (2017)
0,628.0,663.0,788.0,1084.0,1288.0,Alabama,Autauga,"Autauga, Alabama",565.0,647.0,788.0,1029.0,1336.0,565.0,647.0,788.0,1029.0,1336.0
1,494.0,643.0,762.0,1123.0,1318.0,Alabama,Baldwin,"Baldwin, Alabama",719.0,754.0,873.0,1272.0,1472.0,719.0,754.0,873.0,1272.0,1472.0
2,492.0,495.0,670.0,834.0,895.0,Alabama,Barbour,"Barbour, Alabama",472.0,475.0,636.0,789.0,872.0,472.0,475.0,636.0,789.0,872.0
3,545.0,652.0,773.0,1015.0,1142.0,Alabama,Bibb,"Bibb, Alabama",613.0,726.0,840.0,1134.0,1250.0,613.0,726.0,840.0,1134.0,1250.0
4,545.0,652.0,773.0,1015.0,1142.0,Alabama,Blount,"Blount, Alabama",613.0,726.0,840.0,1134.0,1250.0,613.0,726.0,840.0,1134.0,1250.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3134,560.0,669.0,905.0,1127.0,1603.0,Wyoming,Sweetwater,"Sweetwater, Wyoming",642.0,697.0,933.0,1163.0,1629.0,642.0,697.0,933.0,1163.0,1629.0
3135,774.0,921.0,1112.0,1639.0,1696.0,Wyoming,Teton,"Teton, Wyoming",962.0,974.0,1169.0,1682.0,1750.0,962.0,974.0,1169.0,1682.0,1750.0
3136,492.0,496.0,660.0,923.0,1101.0,Wyoming,Uinta,"Uinta, Wyoming",489.0,535.0,711.0,965.0,1217.0,489.0,535.0,711.0,965.0,1217.0
3137,492.0,542.0,643.0,911.0,1096.0,Wyoming,Washakie,"Washakie, Wyoming",453.0,569.0,658.0,959.0,1149.0,453.0,569.0,658.0,959.0,1149.0


In [282]:
unique_merged_counties = sorted(set(merged_dataset.Area).difference(agg_fmr_df.Area.values))
unique_merged_counties2 = sorted(set(agg_fmr_df.Area.values).difference(merged_dataset.Area))

# many are missing from the agg_fmr data set but not the other way around
print("Counties from the merged data set that are missing in the aggregated fmr data set:")
print(unique_merged_counties,"\n\n")
print("Counties from the aggregated fmr data set that are missing in the merged data set:")
print(unique_merged_counties2)


Counties from the merged data set that are missing in the aggregated fmr data set:
['Do�a Ana, New Mexico', 'LaSalle, Illinois', 'LaSalle, Louisiana', 'Oglala Lakota, South Dakota'] 


Counties from the aggregated fmr data set that are missing in the merged data set:
['Adams, Indiana', 'Adams, Mississippi', 'Alcorn, Mississippi', 'Aleutians East, Alaska', 'Aleutians West, Alaska', 'Alexandria, Virginia', 'Allendale, South Carolina', 'Amite, Mississippi', 'Anchorage Municipality, Alaska', 'Armstrong, Pennsylvania', 'Atkinson, Georgia', 'Attala, Mississippi', 'Avery, North Carolina', 'Baker, Georgia', 'Banner, Nebraska', 'Barnstable, Massachusetts', 'Belknap, New Hampshire', 'Benton, Indiana', 'Benton, Mississippi', 'Berkshire, Massachusetts', 'Bertie, North Carolina', 'Bethel, Alaska', 'Bladen, North Carolina', 'Blaine, Nebraska', 'Bolivar, Mississippi', 'Boone, Indiana', 'Boyd, Nebraska', 'Bristol Bay, Alaska', 'Bristol, Massachusetts', 'Bristol, Rhode Island', 'Bristol, Virginia', 'Br

In [283]:
search_for("[Oo]glala|[lL]akota|Shan*n.*Dakota",agg_fmr_df)

,SEARCH,fmr0 (2015),fmr1 (2015),fmr2 (2015),fmr3 (2015),fmr4 (2015),State,County,Area,fmr0 (2016),fmr1 (2016),fmr2 (2016),fmr3 (2016),fmr4 (2016),fmr0 (2017),fmr1 (2017),fmr2 (2017),fmr3 (2017),fmr4 (2017)
2417,[Oo]glala|[lL]akota|Shan*n.*Dakota,468.0,521.0,640.0,851.0,880.0,South Dakota,Shannon,"Shannon, South Dakota",434.0,483.0,644.0,843.0,883.0,434.0,483.0,644.0,843.0,883.0


In [284]:
# we didn't include the modification of Shannon Dakota, because we didn't have population estimate
# data on the county
alt_area_names_dict={'LaSalle, Louisiana':"La.*[sS]alle.*, Louisiana",
                      'LaSalle, Illinois':"La.*[sS]alle.*, Illinois",
                    'Do�a Ana, New Mexico':"Do.*a Ana, New Mexico"}

merged_dataset=merged_dataset.apply(modify_alternate_names,
                                    alt_area_names_dict=alt_area_names_dict,
                                    search_dataset = agg_fmr_df,
                                    axis=1)


LaSalle, Illinois modified to La Salle, Illinois 


LaSalle, Louisiana modified to La Salle, Louisiana 


Do�a Ana, New Mexico modified to Dona Ana, New Mexico 




In [285]:
unique_merged_counties = sorted(set(merged_dataset.Area).difference(agg_fmr_df.Area.values))
unique_merged_counties

['Oglala Lakota, South Dakota']

In [286]:
# merging the fmr info with the merged data set
fmr_year_cols = agg_fmr_df.columns[agg_fmr_df.columns.str.contains("fmr")]
merged_dataset = counties_merge_left(left=merged_dataset, right=agg_fmr_df[["Area"]+fmr_year_cols.tolist()])

fmr_year_cols_2015 = fmr_year_cols[fmr_year_cols.str.contains("fmr[1-4] \(2015\)")]
fmr_year_cols_2016 = fmr_year_cols[fmr_year_cols.str.contains("fmr[1-4] \(2016\)")]
fmr_year_cols_2017 = fmr_year_cols[fmr_year_cols.str.contains("fmr[1-4] \(2017\)")]
merged_dataset["avg_fmr_2017"] = merged_dataset[fmr_year_cols_2017].mean(axis=1)
merged_dataset["avg_fmr_2016"] = merged_dataset[fmr_year_cols_2016].mean(axis=1) 
merged_dataset["avg_fmr_2015"] = merged_dataset[fmr_year_cols_2015].mean(axis=1)
# avg cost to rent an apartment
merged_dataset['avg_fmr_est'] = merged_dataset[['avg_fmr_2015',"avg_fmr_2016","avg_fmr_2017"]].mean(axis=1)
display(merged_dataset[fmr_year_cols_2017].corr())
merged_dataset[fmr_year_cols_2017].describe()

,fmr1 (2017),fmr2 (2017),fmr3 (2017),fmr4 (2017)
fmr1 (2017),1.000000,0.978479,0.954704,0.933341
fmr2 (2017),0.978479,1.000000,0.974582,0.948351
fmr3 (2017),0.954704,0.974582,1.000000,0.957739
fmr4 (2017),0.933341,0.948351,0.957739,1.000000


,fmr1 (2017),fmr2 (2017),fmr3 (2017),fmr4 (2017)
count,2773.000000,2773.000000,2773.000000,2773.000000
mean,591.641147,748.109998,1000.002914,1155.040351
std,149.254520,174.958669,242.207868,306.207614
min,425.000000,569.000000,706.000000,780.000000
25%,497.000000,650.000000,850.000000,951.000000
50%,546.000000,676.000000,925.000000,1071.000000
75%,629.000000,797.000000,1072.000000,1250.000000
max,1814.000000,2289.000000,2987.000000,3556.000000


In [287]:
assert merged_dataset.Area.duplicated().sum() == 0

In [288]:
merged_dataset.columns

Index(['State', 'Raw County Data (2015)', 'Violent crime (2015)',
       'Murder and nonnegligent manslaughter (2015)', 'Rape (revised) (2015)',
       'Rape (previous def) (2015)', 'Robbery (2015)',
       'Aggravated assault (2015)', 'Property crime (2015)', 'Burglary (2015)',
       'Larceny-theft (2015)', 'Motor vehicle theft (2015)', 'Arson (2015)',
       'State/County Type (2015)', 'notice num 1 (2015)',
       'notice num 2 (2015)', 'notice 1 (2015)', 'notice 2 (2015)', 'County',
       'Area', 'Loc Abbrev_x', 'Raw County Data (2016)',
       'Violent crime (2016)', 'Murder and nonnegligent manslaughter (2016)',
       'Rape (revised) (2016)', 'Rape (previous def) (2016)', 'Robbery (2016)',
       'Aggravated assault (2016)', 'Property crime (2016)', 'Burglary (2016)',
       'Larceny-theft (2016)', 'Motor vehicle theft (2016)', 'Arson (2016)',
       'State/County Type (2016)', 'notice num 1 (2016)',
       'notice num 2 (2016)', 'notice 1 (2016)', 'notice 2 (2016)',
       'L

In [289]:
import pandas as pd
incomes=pd.read_excel("/home/sbr0/Dropbox/Analysis Planning/coursera stuffs/Capstone/First-Class Restaurant Location Analysis/data sets/county_income.xlsx")

# from the looks of the data, the rows and column names need re-arranging
display(incomes.head())
display(incomes.tail())

,"Table 1. Per Capita Personal Income by County, 2015 - 2017",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,Per capita personal income1,NaN,NaN,NaN,Percent change from preceding period,NaN,NaN
1,NaN,Dollars,NaN,NaN,Rank in State,Percent change,NaN,Rank in State
2,NaN,2015,2016.0,2017.0,2017,2016,2017.0,2017
3,United States,48940,49831.0,51640.0,--,1.8,3.6,--
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,"Table 1. Per Capita Personal Income by County, 2015 - 2017",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
3218,Washakie,44165,44204.0,45933.0,15,0.1,3.9,6
3219,Weston,47600,42727.0,43340.0,17,-10.2,1.4,19
3220,1. Per capita personal income was computed using Census Bureau midyear population estimates. Estimates reflect count...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3221,"2. Virginia combination areas consist of one or two independent cities with populations of less than 100,000 combine...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3222,Source: U.S. Bureau of Economic Analysis,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [290]:
incomes.columns = ["Raw County Data","Income 2015","Income 2016","Income 2017","Income Rank In State 2017","Rank Percent Change (2015-2016)","Percent Change (2016-2017)","Percent Change Rank In State"]
rows_to_drop = np.arange(5).tolist() + np.arange(3220,3223).tolist()
incomes=incomes.drop(rows_to_drop)

In [291]:
# having the -- marks defines a mid-table value for the state name. The task is to make that into a column rather
# than a section header. Independent City and Combination areas (both specific to Virginia) are also accounted for.
# Because the -- marks are not present in the Independent City or Cobmination areas section headers, we delete those
# rows by default with the following code when dropping all rows with

virginia_ind_cities_idx=incomes.loc[incomes["Raw County Data"].str.contains("Independent cities|Combination areas").fillna(False)].index
virginia_cities_table = incomes.loc[virginia_ind_cities_idx[0]+1:virginia_ind_cities_idx[1]-1]

incomes.loc[incomes["Percent Change Rank In State"] == "--","State"] = incomes.loc[incomes["Percent Change Rank In State"] == "--",'Raw County Data']
incomes['State'] = incomes.State.ffill()
incomes['Independent City'] = incomes.index.isin(virginia_cities_table.index)
incomes = incomes.loc[incomes["Percent Change Rank In State"] != "--"]
incomes=incomes.loc[incomes['Raw County Data'].notnull()].reset_index(drop=True)
incomes['County']=incomes["Raw County Data"].str.replace("St\.","Saint").apply(rm_all_denotations)

In [292]:
assert incomes.State.isin(state_abbreviations.State.values).all()
incomes.sort_values(by=['State',"Income Rank In State 2017"])

,Raw County Data,Income 2015,Income 2016,Income 2017,Income Rank In State 2017,Rank Percent Change (2015-2016),Percent Change (2016-2017),Percent Change Rank In State,State,Independent City,County
58,Shelby,50115,50640.0,52406.0,1,1,3.5,49,Alabama,False,Shelby
36,Jefferson,48902,49678.0,52003.0,2,1.6,4.7,17,Alabama,False,Jefferson
44,Madison,46895,48076.0,49650.0,3,2.5,3.3,57,Alabama,False,Madison
1,Baldwin,41412,43004.0,44079.0,4,3.8,2.5,66,Alabama,False,Baldwin
50,Montgomery,40550,41484.0,43074.0,5,2.3,3.8,35,Alabama,False,Montgomery
...,...,...,...,...,...,...,...,...,...,...,...
3098,Goshen,41436,40553.0,41492.0,19,-2.1,2.3,15,Wyoming,False,Goshen
3102,Lincoln,37492,39501.0,41139.0,20,5.4,4.1,5,Wyoming,False,Lincoln
3091,Albany,39913,39845.0,40301.0,21,-0.2,1.1,21,Wyoming,False,Albany
3111,Uinta,38931,37866.0,38797.0,22,-2.7,2.5,14,Wyoming,False,Uinta


In [293]:
virginia_cities_table

,Raw County Data,Income 2015,Income 2016,Income 2017,Income Rank In State 2017,Rank Percent Change (2015-2016),Percent Change (2016-2017),Percent Change Rank In State
2989,Alexandria,82481,82429.0,84079.0,3,-0.1,2.0,96
2990,Chesapeake,46965,47223.0,48569.0,31,0.5,2.9,79
2991,Hampton,40636,40732.0,42133.0,51,0.2,3.4,48
2992,Newport News,40648,40365.0,41646.0,53,-0.7,3.2,62
2993,Norfolk,38629,38828.0,40094.0,61,0.5,3.3,56
2994,Portsmouth,38766,38461.0,40026.0,62,-0.8,4.1,29
2995,Richmond,51283,51407.0,53581.0,19,0.2,4.2,23
2996,Roanoke,41651,40875.0,41946.0,52,-1.9,2.6,86
2997,Suffolk,48003,48299.0,49779.0,27,0.6,3.1,71
2998,Virginia Beach,52843,53295.0,55108.0,17,0.9,3.4,50


In [294]:
incomes['Area'] = incomes['County'] + ', ' + incomes['State']
missing_locations_no_ind_cities=find_unique_locations(merged_dataset,incomes.loc[incomes['Independent City']==False])
missing_locations=find_unique_locations(merged_dataset,incomes)

print("Total number of unique locations within the merged dataset:",
      len(missing_locations))

print("Total number of unique locations within the merged dataset (excluding independent cities):",
      len(missing_locations_no_ind_cities))

Total number of unique locations within the merged dataset: 28
Total number of unique locations within the merged dataset (excluding independent cities): 29


In [295]:
set(missing_locations_no_ind_cities).difference(missing_locations)

{'Roanoke, Virginia'}

In [296]:
search_for("Roanoke",incomes)

,SEARCH,Raw County Data,Income 2015,Income 2016,Income 2017,Income Rank In State 2017,Rank Percent Change (2015-2016),Percent Change (2016-2017),Percent Change Rank In State,State,Independent City,County,Area
2898,Roanoke,Roanoke,41651,40875.0,41946.0,52,-1.9,2.6,86,Virginia,True,Roanoke,"Roanoke, Virginia"
2917,Roanoke,Roanoke + Salem,48952,48913.0,49860.0,26,-0.1,1.9,98,Virginia,False,Roanoke + Salem,"Roanoke + Salem, Virginia"


In [297]:
search_for("Salem.*Virginia",incomes)

,SEARCH,Raw County Data,Income 2015,Income 2016,Income 2017,Income Rank In State 2017,Rank Percent Change (2015-2016),Percent Change (2016-2017),Percent Change Rank In State,State,Independent City,County,Area
2917,Salem.*Virginia,Roanoke + Salem,48952,48913.0,49860.0,26,-0.1,1.9,98,Virginia,False,Roanoke + Salem,"Roanoke + Salem, Virginia"


In [298]:
search_for("Shan.*n.*Dakota",incomes)

,SEARCH,Raw County Data,Income 2015,Income 2016,Income 2017,Income Rank In State 2017,Rank Percent Change (2015-2016),Percent Change (2016-2017),Percent Change Rank In State,State,Independent City,County,Area


In [299]:
# Drop all rows containing independent cities except for Roanoke, as this city seems to give the closest estimate of income
# in comparison to Roanoke + Salem.
independent_cities_idx=incomes.loc[((incomes['Independent City']==True) & (incomes["County"] != "Roanoke"))].index

print("Columns to drop:")
incomes#=incomes.drop(independent_cities_idx)
incomes.loc[independent_cities_idx]

Columns to drop:


,Raw County Data,Income 2015,Income 2016,Income 2017,Income Rank In State 2017,Rank Percent Change (2015-2016),Percent Change (2016-2017),Percent Change Rank In State,State,Independent City,County,Area
2891,Alexandria,82481,82429.0,84079.0,3,-0.1,2.0,96,Virginia,True,Alexandria,"Alexandria, Virginia"
2892,Chesapeake,46965,47223.0,48569.0,31,0.5,2.9,79,Virginia,True,Chesapeake,"Chesapeake, Virginia"
2893,Hampton,40636,40732.0,42133.0,51,0.2,3.4,48,Virginia,True,Hampton,"Hampton, Virginia"
2894,Newport News,40648,40365.0,41646.0,53,-0.7,3.2,62,Virginia,True,Newport News,"Newport News, Virginia"
2895,Norfolk,38629,38828.0,40094.0,61,0.5,3.3,56,Virginia,True,Norfolk,"Norfolk, Virginia"
2896,Portsmouth,38766,38461.0,40026.0,62,-0.8,4.1,29,Virginia,True,Portsmouth,"Portsmouth, Virginia"
2897,Richmond,51283,51407.0,53581.0,19,0.2,4.2,23,Virginia,True,Richmond,"Richmond, Virginia"
2899,Suffolk,48003,48299.0,49779.0,27,0.6,3.1,71,Virginia,True,Suffolk,"Suffolk, Virginia"
2900,Virginia Beach,52843,53295.0,55108.0,17,0.9,3.4,50,Virginia,True,Virginia Beach,"Virginia Beach, Virginia"


In [300]:
incomes=incomes.drop(independent_cities_idx)

In [301]:
#def split_combined_entries(county_area):
names = ["LaSalle","Lasalle","La salle","La grange","La Grange","Lagrange"]
replacement_names = ["La Salle"]*3 + ["LaGrange"] * 3
area_name_replacement = dict(zip(names,replacement_names))

# used to modify the names of counties in a data set.
def modify_names(area_data,initial_kw="La",replacements=area_name_replacement):
    if initial_kw in area_data.County:
        for name in replacements.keys():
            if name in area_data.County:
                area_data.County = re.sub(name,replacements[name],area_data.County)
                area_data.Area = re.sub(name,replacements[name],area_data.Area)
                break
                
    return area_data

# replaces "+" with ", " and makes the combined county info into "county name, state"
def normalize_combined_county_names(area_data):
    if "+" in area_data['County']:
        area_data.County = area_data['County'].replace(" +",",")
        area_data.Area = area_data['County'] + ", " + area_data['State']
    return area_data

incomes = incomes.apply(modify_names,axis=1)
incomes = incomes.apply(normalize_combined_county_names,axis=1)
missing_county_states = [location.rsplit(", ",1) for location in missing_locations]
location_queries=[county +',.*'+ state for county,state in missing_county_states]

alt_area_names_dict = alt_area_names_dict=search_for("Ana.*New Mexico",incomes).append(search_for("Fre.*Idaho",incomes).replace("(.*","")).set_index("Area")["SEARCH"].squeeze().to_dict()
alt_area_names_dict2=pd.concat([search_for(query,dataset=incomes) for query in location_queries],sort=True).set_index("Area")["SEARCH"].to_dict()
alt_area_names_dict.update(alt_area_names_dict2)

incomes=incomes.apply(modify_alternate_names,alt_area_names_dict=alt_area_names_dict,search_dataset=merged_dataset,axis=1)

Maui, Kalawao, Hawaii modified to Maui, Hawaii 


Fremont (includes Yellowstone National Park), Idaho modified to Fremont, Idaho 


La Salle, Illinois modified to La Salle, Illinois 


LaGrange, Indiana modified to LaGrange, Indiana 


La Salle, Louisiana modified to La Salle, Louisiana 


Doña Ana, New Mexico modified to Dona Ana, New Mexico 


Albemarle, Charlottesville, Virginia modified to Albemarle, Virginia 


Alleghany, Covington, Virginia modified to Alleghany, Virginia 


Augusta, Staunton, Waynesboro, Virginia modified to Augusta, Virginia 


Campbell, Lynchburg, Virginia modified to Campbell, Virginia 


Carroll, Galax, Virginia modified to Carroll, Virginia 


Dinwiddie, Colonial Heights, Petersburg, Virginia modified to Dinwiddie, Virginia 


Fairfax, Fairfax, Virginia modified to Fairfax, Virginia 


Frederick, Winchester, Virginia modified to Frederick, Virginia 


Greensville, Emporia, Virginia modified to Greensville, Virginia 


Henry, Martinsville, Virginia modified 

In [302]:
incomes.County

0          Autauga
1          Baldwin
2          Barbour
3             Bibb
4           Blount
           ...    
3109    Sweetwater
3110         Teton
3111         Uinta
3112      Washakie
3113        Weston
Name: County, Length: 3105, dtype: object

In [303]:
incomes

,Raw County Data,Income 2015,Income 2016,Income 2017,Income Rank In State 2017,Rank Percent Change (2015-2016),Percent Change (2016-2017),Percent Change Rank In State,State,Independent City,County,Area
0,Autauga,38591.0,39509.0,40484.0,9.0,2.4,2.5,67.0,Alabama,False,Autauga,"Autauga, Alabama"
1,Baldwin,41412.0,43004.0,44079.0,4.0,3.8,2.5,66.0,Alabama,False,Baldwin,"Baldwin, Alabama"
2,Barbour,31509.0,31750.0,33453.0,46.0,0.8,5.4,11.0,Alabama,False,Barbour,"Barbour, Alabama"
3,Bibb,29048.0,28989.0,30022.0,66.0,-0.2,3.6,47.0,Alabama,False,Bibb,"Bibb, Alabama"
4,Blount,31987.0,32261.0,33707.0,45.0,0.9,4.5,19.0,Alabama,False,Blount,"Blount, Alabama"
...,...,...,...,...,...,...,...,...,...,...,...,...
3109,Sweetwater,49256.0,47555.0,49870.0,8.0,-3.5,4.9,3.0,Wyoming,False,Sweetwater,"Sweetwater, Wyoming"
3110,Teton,204913.0,224366.0,233860.0,1.0,9.5,4.2,4.0,Wyoming,False,Teton,"Teton, Wyoming"
3111,Uinta,38931.0,37866.0,38797.0,22.0,-2.7,2.5,14.0,Wyoming,False,Uinta,"Uinta, Wyoming"
3112,Washakie,44165.0,44204.0,45933.0,15.0,0.1,3.9,6.0,Wyoming,False,Washakie,"Washakie, Wyoming"


In [304]:
assert not find_unique_locations(merged_dataset.loc[excluding_shannon],incomes)

In [305]:
#assert not find_unique_locations(merged_dataset,incomes)


# Richmond is duplicated with independent city data that we thus delete
cities_idx = incomes.loc[incomes['Raw County Data'].str.contains("[cC]ity")].index.tolist()


#Below, the entries to exclude are shown:

incomes_no_cty=incomes.drop(cities_idx)
incomes_no_cty.loc[incomes_no_cty.Area.duplicated()].empty

unique_wo_cities = find_unique_locations(merged_dataset,incomes_no_cty)
unique_w_cities = find_unique_locations(merged_dataset,incomes)

print("Difference:",set(unique_w_cities).difference(unique_wo_cities))

Difference: set()


In [306]:
# It turns out  that a city is not the same as an independent city:
incomes.iloc[cities_idx]

,Raw County Data,Income 2015,Income 2016,Income 2017,Income Rank In State 2017,Rank Percent Change (2015-2016),Percent Change (2016-2017),Percent Change Rank In State,State,Independent City,County,Area
77,Juneau City and Borough,63861.0,64319.0,66367.0,5.0,0.7,3.2,12.0,Alaska,False,Juneau,"Juneau, Alaska"
89,Sitka City and Borough,67048.0,61931.0,65745.0,6.0,-7.6,6.2,3.0,Alaska,False,Sitka,"Sitka, Alaska"
93,Wrangell City and Borough,47287.0,44748.0,44286.0,26.0,-5.4,-1.0,26.0,Alaska,False,Wrangell,"Wrangell, Alaska"
94,Yakutat City and Borough,60614.0,53897.0,52812.0,20.0,-11.1,-2.0,27.0,Alaska,False,Yakutat,"Yakutat, Alaska"
1214,Baltimore City,46150.0,47553.0,49168.0,15.0,3.0,3.4,14.0,Maryland,False,Baltimore,"Baltimore, Maryland"
1595,St. Louis City,42210.0,42490.0,43577.0,11.0,0.7,2.6,82.0,Missouri,False,Saint Louis,"Saint Louis, Missouri"
1761,Carson City,45488.0,45219.0,48997.0,5.0,-0.6,8.4,3.0,Nevada,False,Carson,"Carson, Nevada"
2831,Charles City,40693.0,41495.0,42788.0,48.0,2.0,3.1,67.0,Virginia,False,Charles,"Charles, Virginia"
2917,Roanoke + Salem,48952.0,48913.0,49860.0,26.0,-0.1,1.9,98.0,Virginia,False,"Roanoke, Salem","Roanoke, Salem, Virginia"
2921,Spotsylvania + Fredericksburg,46368.0,47318.0,48823.0,29.0,2.0,3.2,61.0,Virginia,False,Spotsylvania,"Spotsylvania, Virginia"


In [307]:
# Appears to contains both Fairfax County and City Data:
search_for("Fairfax",incomes)

,SEARCH,Raw County Data,Income 2015,Income 2016,Income 2017,Income Rank In State 2017,Rank Percent Change (2015-2016),Percent Change (2016-2017),Percent Change Rank In State,State,Independent City,County,Area
2908,Fairfax,"Fairfax, Fairfax City + Falls Church",75756.0,76213.0,78376.0,4.0,0.6,2.8,80.0,Virginia,False,Fairfax,"Fairfax, Virginia"


In [308]:
incomes.loc[incomes.Area == "Fairfax, Virginia"].index

Int64Index([2908], dtype='int64')

In [309]:
cities_idx.remove(search_for("Fairfax",incomes).index)
relevant_columns = ["Area","Income 2015","Income 2016","Income 2017"]
merged_dataset = pd.merge(merged_dataset,incomes[relevant_columns].loc[~(incomes.index.isin(cities_idx))],
                          left_on="Area",right_on="Area",how='left')
assert merged_dataset.Area.duplicated().sum() == 0
merged_dataset
                          

,State,Raw County Data (2015),Violent crime (2015),Murder and nonnegligent manslaughter (2015),Rape (revised) (2015),Rape (previous def) (2015),Robbery (2015),Aggravated assault (2015),Property crime (2015),Burglary (2015),...,fmr4 (2016),fmr0 (2017),fmr1 (2017),fmr2 (2017),fmr3 (2017),fmr4 (2017),avg_fmr_2017,Income 2015,Income 2016,Income 2017
0,Alabama,Autauga,69.0,0.0,13.0,NaN,6.0,50.0,344.0,111.0,...,1336.0,565.0,647.0,788.0,1029.0,1336.0,950.00,38591.0,39509.0,40484.0
1,Alabama,Baldwin,115.0,0.0,9.0,NaN,33.0,73.0,648.0,225.0,...,1472.0,719.0,754.0,873.0,1272.0,1472.0,1092.75,41412.0,43004.0,44079.0
2,Alabama,Bibb,7.0,0.0,2.0,NaN,1.0,4.0,41.0,20.0,...,1250.0,613.0,726.0,840.0,1134.0,1250.0,987.50,29048.0,28989.0,30022.0
3,Alabama,Blount,204.0,5.0,16.0,NaN,5.0,178.0,832.0,247.0,...,1250.0,613.0,726.0,840.0,1134.0,1250.0,987.50,31987.0,32261.0,33707.0
4,Alabama,Calhoun,16.0,0.0,4.0,NaN,1.0,11.0,413.0,181.0,...,896.0,445.0,476.0,637.0,810.0,896.0,704.75,34188.0,34869.0,36255.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2769,Vermont,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1042.0,587.0,612.0,760.0,943.0,1042.0,839.25,33287.0,33556.0,34997.0
2770,Vermont,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1444.0,685.0,728.0,920.0,1176.0,1444.0,1067.00,43146.0,43925.0,45275.0
2771,Vermont,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1479.0,732.0,737.0,986.0,1224.0,1479.0,1106.50,54438.0,56711.0,58968.0
2772,Vermont,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1671.0,812.0,848.0,1041.0,1420.0,1671.0,1245.00,52417.0,53359.0,55034.0


In [310]:
# gives median per capita income data as well as unemployment/employment rates
affluence_stats = pd.read_csv("/home/sbr0/Dropbox/Analysis Planning/coursera stuffs/Capstone/First-Class Restaurant Location Analysis/data sets/Employment and Median Household Income.csv")
affluence_stats=affluence_stats.rename(columns = affluence_stats.iloc[6]).drop([0,1,2,3,4,5,6]).reset_index(drop=True)
display(affluence_stats.head(10))
display(affluence_stats.tail(10))

,FIPStxt,State,Area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2007,Employed_2007,Unemployed_2007,Unemployment_rate_2007,...,Civilian_labor_force_2016,Employed_2016,Unemployed_2016,Unemployment_rate_2016,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017,Median_Household_Income_2016,Med_HH_Income_Percent_of_State_Total_2016
0,00000,US,United States,NaN,NaN,NaN,"152,191,093","145,156,134","7,034,959",4.6,...,"158,921,892","151,183,680","7,738,212",4.9,"160,588,515","153,594,100","6,994,415",4.4,"$57,617",NaN
1,01000,AL,Alabama,NaN,NaN,NaN,"2,175,612","2,089,127","86,485",4.0,...,"2,173,175","2,045,624","127,551",5.9,2168444,2073106,95338,4.4,"$46,309",100.0
2,01001,AL,"Autauga County, AL",2,2,1,"24,383","23,577",806,3.3,...,"25,918","24,593","1,325",5.1,25909,24908,1001,3.9,"$54,487",117.7
3,01003,AL,"Baldwin County, AL",3,2,1,"82,659","80,099","2,560",3.1,...,"90,500","85,656","4,844",5.4,91567,87915,3652,4.0,"$56,460",121.9
4,01005,AL,"Barbour County, AL",6,6,0,"10,334","9,684",650,6.3,...,"8,402","7,700",702,8.4,8236,7750,486,5.9,"$32,884",71.0
5,01007,AL,"Bibb County, AL",1,1,1,"8,791","8,432",359,4.1,...,"8,607","8,050",557,6.5,8506,8133,373,4.4,"$43,079",93.0
6,01009,AL,"Blount County, AL",1,1,1,"26,629","25,780",849,3.2,...,"24,576","23,248","1,328",5.4,24494,23509,985,4.0,"$47,213",102.0
7,01011,AL,"Bullock County, AL",6,6,0,"3,653","3,308",345,9.4,...,"4,824","4,493",331,6.9,4812,4575,237,4.9,"$34,278",74.0
8,01013,AL,"Butler County, AL",6,6,0,"9,099","8,539",560,6.2,...,"9,190","8,553",637,6.9,8946,8455,491,5.5,"$35,409",76.5
9,01015,AL,"Calhoun County, AL",3,2,1,"54,861","52,709","2,152",3.9,...,"45,818","42,815","3,003",6.6,45582,43333,2249,4.9,"$41,778",90.2


,FIPStxt,State,Area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2007,Employed_2007,Unemployed_2007,Unemployment_rate_2007,...,Civilian_labor_force_2016,Employed_2016,Unemployed_2016,Unemployment_rate_2016,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017,Median_Household_Income_2016,Med_HH_Income_Percent_of_State_Total_2016
3265,72135,PR,"Toa Alta Municipio, PR",1,1,1,"30,318","27,813","2,505",8.3,...,"26,588","24,374","2,214",8.3,26362,24304,2058,7.8,NaN,NaN
3266,72137,PR,"Toa Baja Municipio, PR",1,1,1,"36,123","32,909","3,214",8.9,...,"29,794","27,105","2,689",9.0,29402,27033,2369,8.1,NaN,NaN
3267,72139,PR,"Trujillo Alto Municipio, PR",1,1,1,"35,625","32,943","2,682",7.5,...,"27,164","25,206","1,958",7.2,26734,25158,1576,5.9,NaN,NaN
3268,72141,PR,"Utuado Municipio, PR",2,2,1,"9,177","7,817","1,360",14.8,...,"6,895","5,824","1,071",15.5,6860,5908,952,13.9,NaN,NaN
3269,72143,PR,"Vega Alta Municipio, PR",1,1,1,"12,663","11,235","1,428",11.3,...,"9,892","8,751","1,141",11.5,9774,8723,1051,10.8,NaN,NaN
3270,72145,PR,"Vega Baja Municipio, PR",1,1,1,"21,115","18,445","2,670",12.6,...,"13,753","11,844","1,909",13.9,13467,11795,1672,12.4,NaN,NaN
3271,72147,PR,"Vieques Municipio, PR",7,12,0,"3,317","2,955",362,10.9,...,"3,263","2,917",346,10.6,3228,2787,441,13.7,NaN,NaN
3272,72149,PR,"Villalba Municipio, PR",2,2,1,"9,479","8,223","1,256",13.3,...,"7,827","6,243","1,584",20.2,7631,6138,1493,19.6,NaN,NaN
3273,72151,PR,"Yabucoa Municipio, PR",1,1,1,"11,425","9,348","2,077",18.2,...,"9,095","7,555","1,540",16.9,9010,7533,1477,16.4,NaN,NaN
3274,72153,PR,"Yauco Municipio, PR",2,2,1,"16,635","14,451","2,184",13.1,...,"10,752","8,731","2,021",18.8,10276,8498,1778,17.3,NaN,NaN


In [311]:
affluence_stats = affluence_stats.rename(columns={"State":"state_alpha","Area_name":"raw_area_name"})
affluence_stats['State'] = affluence_stats.state_alpha.map(state_abbreviations.set_index("Abbreviation").to_dict()['State'])

In [312]:
# drop rows denoting a change in reporting for information in subsequent columns and nothing else
states_and_country_idx=affluence_stats.loc[(affluence_stats.raw_area_name.isin(state_abbreviations.State)) |
                   (affluence_stats.raw_area_name == 'United States')].index
affluence_stats = affluence_stats.drop(states_and_country_idx)
affluence_stats.head()

,FIPStxt,state_alpha,raw_area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2007,Employed_2007,Unemployed_2007,Unemployment_rate_2007,...,Employed_2016,Unemployed_2016,Unemployment_rate_2016,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017,Median_Household_Income_2016,Med_HH_Income_Percent_of_State_Total_2016,State
2,01001,AL,"Autauga County, AL",2,2,1,"24,383","23,577",806,3.3,...,"24,593","1,325",5.1,25909,24908,1001,3.9,"$54,487",117.7,Alabama
3,01003,AL,"Baldwin County, AL",3,2,1,"82,659","80,099","2,560",3.1,...,"85,656","4,844",5.4,91567,87915,3652,4.0,"$56,460",121.9,Alabama
4,01005,AL,"Barbour County, AL",6,6,0,"10,334","9,684",650,6.3,...,"7,700",702,8.4,8236,7750,486,5.9,"$32,884",71.0,Alabama
5,01007,AL,"Bibb County, AL",1,1,1,"8,791","8,432",359,4.1,...,"8,050",557,6.5,8506,8133,373,4.4,"$43,079",93.0,Alabama
6,01009,AL,"Blount County, AL",1,1,1,"26,629","25,780",849,3.2,...,"23,248","1,328",5.4,24494,23509,985,4.0,"$47,213",102.0,Alabama


In [313]:
def strip_state_name(area):
    abbrev_substr = (", " + state_abbreviations.Abbreviation).tolist()
    contains_state_code = re.search("|".join(abbrev_substr),area)
    if contains_state_code is not None:
        code=contains_state_code.group()
        area = area.replace(code,"")
    return area

# removes all state abbreviations for the sake of having a county name only column
# while standardizing the presentation of the county column
affluence_stats['County']=(
    affluence_stats.raw_area_name
 .str.replace("St\.","Saint")
 .str.replace("Municipio.*","")
 .apply(rm_all_denotations)
 .apply(strip_state_name)
 .str.strip()
)

In [314]:
affluence_stats['Area'] = affluence_stats.County + ', ' + affluence_stats.State
affluence_stats['Loc Abbrev'] = affluence_stats['County'].str[0] + ', ' + affluence_stats.State
affluence_stats[["Area","Loc Abbrev"]]

,Area,Loc Abbrev
2,"Autauga, Alabama","A, Alabama"
3,"Baldwin, Alabama","B, Alabama"
4,"Barbour, Alabama","B, Alabama"
5,"Bibb, Alabama","B, Alabama"
6,"Blount, Alabama","B, Alabama"
...,...,...
3270,NaN,NaN
3271,NaN,NaN
3272,NaN,NaN
3273,NaN,NaN


In [315]:
# drops areas that are duplicated in the affluence information
duped_areas=affluence_stats.Area.loc[(affluence_stats.Area.duplicated()) & (affluence_stats.Area.notnull())].values
dupe_area_city_idx = affluence_stats.loc[(affluence_stats.Area.isin(duped_areas)) & (affluence_stats.raw_area_name.str.contains("[cC]ity"))].index
affluence_stats = affluence_stats.drop(dupe_area_city_idx)

In [316]:
# areas that contain "city" or "City" in their name
affluence_stats.loc[affluence_stats.raw_area_name.str.contains("[cC]ity")]

,FIPStxt,state_alpha,raw_area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2007,Employed_2007,Unemployed_2007,Unemployment_rate_2007,...,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017,Median_Household_Income_2016,Med_HH_Income_Percent_of_State_Total_2016,State,County,Area,Loc Abbrev
80,02110,AK,"Juneau Borough/city, AK",5,8,0,"18,459","17,626",833,4.5,...,17435,16607,828,4.7,"$82,011",107.7,Alaska,Juneau,"Juneau, Alaska","J, Alaska"
93,02220,AK,"Sitka Borough/city, AK",7,11,0,"4,635","4,398",237,5.1,...,4449,4241,208,4.7,"$72,795",95.6,Alaska,Sitka,"Sitka, Alaska","S, Alaska"
98,02275,AK,"Wrangell Borough/city, AK",9,12,0,NaN,NaN,NaN,NaN,...,1050,969,81,7.7,"$56,747",74.5,Alaska,Wrangell,"Wrangell, Alaska","W, Alaska"
100,02282,AK,"Yakutat Borough/city, AK",9,12,0,308,287,21,6.8,...,235,213,22,9.4,"$57,687",75.8,Alaska,Yakutat,"Yakutat, Alaska","Y, Alaska"
232,06075,CA,"San Francisco County/city, CA",1,1,1,"432,564","414,199","18,365",4.2,...,568722,552223,16499,2.9,"$101,873",150.4,California,San Francisco,"San Francisco, California","S, California"
261,08014,CO,"Broomfield County/city, CO",1,1,1,"29,984","28,928","1,056",3.5,...,37971,36974,997,2.6,"$86,870",132.2,Colorado,Broomfield,"Broomfield, Colorado","B, Colorado"
270,08031,CO,"Denver County/city, CO",1,1,1,"314,781","301,750","13,031",4.1,...,397166,386225,10941,2.8,"$61,038",92.9,Colorado,Denver,"Denver, Colorado","D, Colorado"
563,15003,HI,"Honolulu County/city, HI",2,2,1,"442,512","430,846","11,666",2.6,...,471330,460776,10554,2.2,"$80,012",107.2,Hawaii,Honolulu,"Honolulu, Hawaii","H, Hawaii"
1795,32510,NV,"Carson City, NV",3,2,1,"28,410","27,047","1,363",4.8,...,25610,24316,1294,5.1,"$52,253",94.7,Nevada,Carson,"Carson, Nevada","C, Nevada"
2336,42101,PA,"Philadelphia County/city, PA",1,1,1,"620,518","582,727","37,791",6.1,...,704053,660073,43980,6.2,"$41,514",73.0,Pennsylvania,Philadelphia,"Philadelphia, Pennsylvania","P, Pennsylvania"


In [317]:
find_unique_locations(merged_dataset,affluence_stats)

['La Salle, Illinois', 'La Salle, Louisiana', 'Shannon, South Dakota']

In [318]:
search_for("Sh.*n.*n.*Dakota",affluence_stats)
#affluence_stats.loc[affluence_stats.Area.str.contains("Shannon").fillna(False)]

,SEARCH,FIPStxt,state_alpha,raw_area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2007,Employed_2007,Unemployed_2007,...,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017,Median_Household_Income_2016,Med_HH_Income_Percent_of_State_Total_2016,State,County,Area,Loc Abbrev


In [319]:
affluence_stats.Area=affluence_stats.Area.str.replace("La.*Salle", "La Salle")
affluence_stats.County=affluence_stats.County.str.replace("La.*Salle", "La Salle")
city_idx=affluence_stats.loc[affluence_stats.raw_area_name.str.contains('[cC]ity')].index
affluence_stats=affluence_stats.drop(city_idx)

In [320]:
# assert that all locations in the merged data set can be found in the affluence_stats data set
assert not find_unique_locations(merged_dataset[excluding_shannon],affluence_stats)


In [321]:
# double checks whether any row "raw_area_name" is actually a state
affluence_stats.raw_area_name.isin(state_abbreviations.State.values).sum()

0

In [322]:
relevant_columns = [col for col in affluence_stats.columns if "Unemployed" in col or "Employed" in col or "Area" in col]

In [323]:
# merges employment info and median income data with the existing merged data set
merged_dataset=pd.merge(merged_dataset,affluence_stats[relevant_columns],
                        left_on="Area",right_on="Area",how='left')

In [324]:
assert merged_dataset.Area.dropna().duplicated().sum() == 0

In [325]:
merged_dataset.isnull().sum()

State                                            0
Raw County Data (2015)                         212
Violent crime (2015)                           212
Murder and nonnegligent manslaughter (2015)    212
Rape (revised) (2015)                          597
                                              ... 
 Unemployed_2015                                 1
 Employed_2016                                   1
 Unemployed_2016                                 1
Employed_2017                                    1
Unemployed_2017                                  1
Length: 106, dtype: int64

In [326]:
# gets income and job related columns
income_cols=merged_dataset.columns[merged_dataset.columns.str.contains("[Ii]ncome|[fF][mM][rR]|Employed")]
merged_dataset.loc[:,income_cols].dtypes

fmr0 (2015)        float64
fmr1 (2015)        float64
fmr2 (2015)        float64
fmr3 (2015)        float64
fmr4 (2015)        float64
fmr0 (2016)        float64
fmr1 (2016)        float64
fmr2 (2016)        float64
fmr3 (2016)        float64
fmr4 (2016)        float64
fmr0 (2017)        float64
fmr1 (2017)        float64
fmr2 (2017)        float64
fmr3 (2017)        float64
fmr4 (2017)        float64
avg_fmr_2017       float64
Income 2015        float64
Income 2016        float64
Income 2017        float64
 Employed_2007      object
 Employed_2008      object
Employed_2009       object
 Employed_2010      object
 Employed_2011      object
 Employed_2012      object
 Employed_2013      object
 Employed_2014      object
 Employed_2015      object
 Employed_2016      object
Employed_2017       object
dtype: object

In [327]:
work_cols=merged_dataset.columns[merged_dataset.columns.str.contains("mployed")]

def parse_float(variable):
    try:
        return variable.astype(float)
    except ValueError:
        return variable.str.replace(",","").str.strip().astype(float)

    # removes commas from the name of the column:
merged_dataset[work_cols]=merged_dataset[work_cols].apply(parse_float)

In [328]:
merged_dataset[income_cols].corr() # pearson's correlation between each of these columns

,fmr0 (2015),fmr1 (2015),fmr2 (2015),fmr3 (2015),fmr4 (2015),fmr0 (2016),fmr1 (2016),fmr2 (2016),fmr3 (2016),fmr4 (2016),...,Employed_2008,Employed_2009,Employed_2010,Employed_2011,Employed_2012,Employed_2013,Employed_2014,Employed_2015,Employed_2016,Employed_2017
fmr0 (2015),1.000000,0.935184,0.923636,0.893691,0.870908,0.925795,0.899536,0.889435,0.861079,0.850735,...,0.368200,0.369816,0.370164,0.370643,0.370797,0.370013,0.368838,0.369108,0.368540,0.368014
fmr1 (2015),0.935184,1.000000,0.979269,0.950889,0.921529,0.902079,0.953318,0.945876,0.918810,0.898016,...,0.417057,0.418751,0.419227,0.419717,0.419883,0.419059,0.417954,0.418386,0.417916,0.417426
fmr2 (2015),0.923636,0.979269,1.000000,0.968008,0.934721,0.893575,0.942260,0.961945,0.935426,0.910243,...,0.424011,0.425724,0.425875,0.426311,0.426514,0.425787,0.424763,0.425190,0.424657,0.424178
fmr3 (2015),0.893691,0.950889,0.968008,1.000000,0.944013,0.867463,0.917144,0.936959,0.946838,0.915076,...,0.421035,0.422844,0.422907,0.423384,0.423734,0.423218,0.422459,0.423086,0.422897,0.422844
fmr4 (2015),0.870908,0.921529,0.934721,0.944013,1.000000,0.850384,0.891376,0.908299,0.912751,0.931201,...,0.402529,0.404500,0.404859,0.405473,0.405969,0.405573,0.404983,0.405630,0.405639,0.405864
fmr0 (2016),0.925795,0.902079,0.893575,0.867463,0.850384,1.000000,0.943416,0.931111,0.909521,0.898608,...,0.374908,0.376736,0.377728,0.378574,0.378930,0.378196,0.377231,0.377579,0.377262,0.377020
fmr1 (2016),0.899536,0.953318,0.942260,0.917144,0.891376,0.943416,1.000000,0.979356,0.956681,0.936181,...,0.416285,0.418195,0.419314,0.420105,0.420413,0.419594,0.418655,0.419154,0.418850,0.418539
fmr2 (2016),0.889435,0.945876,0.961945,0.936959,0.908299,0.931111,0.979356,1.000000,0.975770,0.950408,...,0.426872,0.428803,0.429576,0.430306,0.430646,0.429904,0.429037,0.429532,0.429163,0.428827
fmr3 (2016),0.861079,0.918810,0.935426,0.946838,0.912751,0.909521,0.956681,0.975770,1.000000,0.959417,...,0.427260,0.429302,0.429973,0.430737,0.431213,0.430659,0.430027,0.430716,0.430682,0.430751
fmr4 (2016),0.850735,0.898016,0.910243,0.915076,0.931201,0.898608,0.936181,0.950408,0.959417,1.000000,...,0.403248,0.405497,0.406731,0.407726,0.408317,0.407875,0.407414,0.408100,0.408290,0.408672


In [329]:
# for merging geographical information pertaining to each county
land_area = pd.read_csv('/home/sbr0/Dropbox/Analysis Planning/coursera stuffs/Capstone/First-Class Restaurant Location Analysis/county population data/land area.csv',encoding="Windows-1252")
land_area.head()

,GEO.id,GEO.id2,GEO.display-label,GCT_STUB.target-geo-id,GCT_STUB.target-geo-id2,GCT_STUB.display-label,GCT_STUB.display-label.1,HD01,HD02,SUBHD0301,SUBHD0302,SUBHD0303,SUBHD0401,SUBHD0402
0,Id,Id2,Geography,Target Geo Id,Target Geo Id2,Geographic area,Geographic area,Population,Housing units,Area in square miles - Total area,Area in square miles - Water area,Area in square miles - Land area,Density per square mile of land area - Population,Density per square mile of land area - Housing units
1,0100000US,NaN,United States,0100000US,NaN,United States,United States,308745538(r38234),131704730(r15031),3796742.23,264836.79,3531905.43,87.4,37.3
2,0100000US,NaN,United States,0400000US01,01,United States - Alabama,Alabama,4779736(r38235),2171853(r15032),52420.07,1774.74,50645.33,94.4,42.9
3,0100000US,NaN,United States,0500000US01001,01001,United States - Alabama - Autauga County,Autauga County,54571,22135,604.39,9.95,594.44,91.8,37.2
4,0100000US,NaN,United States,0500000US01003,01003,United States - Alabama - Baldwin County,Baldwin County,182265,104061,2027.31,437.53,1589.78,114.6,65.5


In [330]:
# the area columns are in the first row:
headers_dict = dict(zip(land_area.columns,land_area.loc[0].values))
headers_dict["GCT_STUB.display-label"] = "raw_area_data"
headers_dict['GCT_STUB.display-label.1'] = 'Raw County Data'
headers_dict['SUBHD0402'] = 'Housing Density (per sq mi)'
headers_dict['SUBHD0301'] = 'Geographical Area (sq mi)'
land_area = land_area.rename(columns=headers_dict).drop(0)

In [331]:
state_country_idx=land_area.loc[(land_area["Raw County Data"].isin(state_abbreviations.State) | (land_area['Raw County Data'] == 'United States'))].index
len(state_country_idx) 

53

In [332]:
land_area = land_area.drop(state_country_idx)

In [333]:
# extracts the state name from the "raw_area_data" column
states_regex = "(" + "|".join(state_abbreviations.State.tolist()) + ")"
land_area['State'] = land_area.raw_area_data.str.extract(states_regex)#

In [334]:
# asserts that all columns in the merged data set can be found in the land data set
assert merged_dataset.State.isin(land_area.State.values).all()
missing_indices = land_area.loc[land_area['State'].isnull()].index

In [335]:
land_area = land_area.drop(missing_indices)

In [336]:
land_area['County']=land_area['Raw County Data'].str.replace("St\.","Saint").apply(rm_all_denotations)

In [337]:
land_area['Area'] = land_area['County'] + ', ' + land_area["State"]

In [338]:
land_area['Area'].duplicated().sum()

7

In [339]:
# shows duplicated area names:
duplicated_areas = land_area['Area'].loc[land_area['Area'].duplicated()]
land_area.loc[land_area['Area'].isin(duplicated_areas)].sort_values(by="Area")

,Id,Id2,Geography,Target Geo Id,Target Geo Id2,raw_area_data,Raw County Data,Population,Housing units,Geographical Area (sq mi),Area in square miles - Water area,Area in square miles - Land area,Density per square mile of land area - Population,Housing Density (per sq mi),State,County,Area
1218,0100000US,NaN,United States,0500000US24005,24005,United States - Maryland - Baltimore County,Baltimore County,805029(r42279),335622(r18612),681.76,83.45,598.30,1345.5,561.0,Maryland,Baltimore,"Baltimore, Maryland"
1239,0100000US,NaN,United States,0500000US24510,24510,United States - Maryland - Baltimore city,Baltimore city,620961(r42352),296685(r18689),92.05,11.11,80.94,7671.5,3665.3,Maryland,Baltimore,"Baltimore, Maryland"
2878,0100000US,NaN,United States,0500000US51019,51019,United States - Virginia - Bedford County,Bedford County,68676,31937,769.25,16.23,753.02,91.2,42.4,Virginia,Bedford,"Bedford, Virginia"
2965,0100000US,NaN,United States,0500000US51515,51515,United States - Virginia - Bedford city,Bedford city,6222,2920,6.90,0.02,6.88,904.6,424.5,Virginia,Bedford,"Bedford, Virginia"
2897,0100000US,NaN,United States,0500000US51059,51059,United States - Virginia - Fairfax County,Fairfax County,1081726(r49532),407998(r25516),406.25,15.28,390.97,2766.8,1043.6,Virginia,Fairfax,"Fairfax, Virginia"
2974,0100000US,NaN,United States,0500000US51600,51600,United States - Virginia - Fairfax city,Fairfax city,22565,8680,6.28,0.04,6.24,3616.6,1391.2,Virginia,Fairfax,"Fairfax, Virginia"
2901,0100000US,NaN,United States,0500000US51067,51067,United States - Virginia - Franklin County,Franklin County,56159,29315,711.55,21.12,690.43,81.3,42.5,Virginia,Franklin,"Franklin, Virginia"
2976,0100000US,NaN,United States,0500000US51620,51620,United States - Virginia - Franklin city,Franklin city,8582,3901,8.35,0.14,8.21,1045.8,475.4,Virginia,Franklin,"Franklin, Virginia"
2944,0100000US,NaN,United States,0500000US51159,51159,United States - Virginia - Richmond County,Richmond County,9254,3850,216.36,24.87,191.49,48.3,20.1,Virginia,Richmond,"Richmond, Virginia"
2994,0100000US,NaN,United States,0500000US51760,51760,United States - Virginia - Richmond city,Richmond city,204214,98349,62.46,2.66,59.81,3414.7,1644.5,Virginia,Richmond,"Richmond, Virginia"


In [340]:
# removes duplicates where city info is provided:
duped_names_of_cities_idx = land_area.loc[(land_area['Area'].isin(duplicated_areas)) & (land_area['raw_area_data'].str.contains("[cC]ity"))].index
land_area=land_area.drop(duped_names_of_cities_idx)

In [341]:
# drops city information
cities_idx=land_area.loc[land_area['raw_area_data'].str.contains("[cC]ity")].index
land_area=land_area.drop(cities_idx)

In [342]:
search_for("Shannon, .* Dakota",land_area)

,SEARCH,Id,Id2,Geography,Target Geo Id,Target Geo Id2,raw_area_data,Raw County Data,Population,Housing units,Geographical Area (sq mi),Area in square miles - Water area,Area in square miles - Land area,Density per square mile of land area - Population,Housing Density (per sq mi),State,County,Area
2461,"Shannon, .* Dakota",0100000US,NaN,United States,0500000US46113,46113,United States - South Dakota - Shannon County,Shannon County,13586,3593,2096.65,2.75,2093.90,6.5,1.7,South Dakota,Shannon,"Shannon, South Dakota"


In [343]:
# finds locations not in land area data which excludes cities
find_unique_locations(merged_dataset,land_area.loc[~land_area['raw_area_data'].str.contains("[cC]ity")])

['Dona Ana, New Mexico', 'La Salle, Illinois', 'Oglala Lakota, South Dakota']

In [344]:
unique_locations=["Ana, New Mexico","Salle, Illinois","Oglala","Lakota, South Dakota"]

alt_area_names_dict=alt_area_names_dict=pd.concat([search_for(area_name,land_area).set_index("Area")['SEARCH'] for area_name in unique_locations])

land_area=land_area.apply(modify_alternate_names,alt_area_names_dict=alt_area_names_dict,search_dataset=merged_dataset,axis=1)

LaSalle, Illinois modified to La Salle, Illinois 


Doña Ana, New Mexico modified to Dona Ana, New Mexico 




In [345]:
unique_locations=find_unique_locations(merged_dataset,land_area.loc[~land_area['raw_area_data'].str.contains("[cC]ity")])
unique_omiting_oglagla = [loc for loc in unique_locations if "Oglala Lakota, South Dakota" != loc]
assert len(unique_omiting_oglagla) == 0, "At least 1 County within the original dataset was not found in the geographical dataset"

In [346]:
relevant_columns = ['Area',"Geographical Area (sq mi)",'Housing Density (per sq mi)',"Population"]
merged_dataset =  pd.merge(merged_dataset,land_area[relevant_columns],left_on='Area',right_on='Area',how="left")
assert merged_dataset.duplicated().sum() == 0

In [347]:
merged_dataset.columns

Index(['State', 'Raw County Data (2015)', 'Violent crime (2015)',
       'Murder and nonnegligent manslaughter (2015)', 'Rape (revised) (2015)',
       'Rape (previous def) (2015)', 'Robbery (2015)',
       'Aggravated assault (2015)', 'Property crime (2015)', 'Burglary (2015)',
       ...
       ' Unemployed_2014 ', ' Employed_2015 ', ' Unemployed_2015 ',
       ' Employed_2016 ', ' Unemployed_2016 ', 'Employed_2017',
       'Unemployed_2017', 'Geographical Area (sq mi)',
       'Housing Density (per sq mi)', 'Population'],
      dtype='object', length=109)

In [348]:
assert merged_dataset.Area.duplicated().sum() == 0

In [354]:
# most of the information pertaining to these two counties was missing. These were dropped from the final
# data set:
idx_to_drop = merged_dataset.loc[merged_dataset.Area.isin(("Oglala Lakota, South Dakota","Shannon, South Dakota"))].index # had few observed values for county information/ average income was below the poverty line
merged_dataset = merged_dataset.drop(idx_to_drop)

In [350]:
merged_dataset.corr()

,Violent crime (2015),Murder and nonnegligent manslaughter (2015),Rape (revised) (2015),Rape (previous def) (2015),Robbery (2015),Aggravated assault (2015),Property crime (2015),Burglary (2015),Larceny-theft (2015),Motor vehicle theft (2015),...,Employed_2013,Unemployed_2013,Employed_2014,Unemployed_2014,Employed_2015,Unemployed_2015,Employed_2016,Unemployed_2016,Employed_2017,Unemployed_2017
Violent crime (2015),1.000000,0.912615,0.859724,0.534713,0.941810,0.988143,0.943053,0.935141,0.910503,0.920939,...,0.613063,0.565990,0.614066,0.578827,0.612355,0.599091,0.611273,0.619764,0.613073,0.628020
Murder and nonnegligent manslaughter (2015),0.912615,1.000000,0.769416,0.476129,0.878822,0.888892,0.877111,0.867976,0.841860,0.889442,...,0.626019,0.606828,0.626873,0.622093,0.625724,0.636717,0.625769,0.639510,0.627768,0.646936
Rape (revised) (2015),0.859724,0.769416,1.000000,-0.316507,0.720658,0.868255,0.835183,0.818096,0.823671,0.735749,...,0.553278,0.492741,0.554278,0.503287,0.553024,0.520615,0.552866,0.536571,0.554789,0.542490
Rape (previous def) (2015),0.534713,0.476129,-0.316507,1.000000,0.559738,0.500956,0.550437,0.732039,0.461689,0.604025,...,0.411604,0.478372,0.414099,0.489991,0.416203,0.488146,0.422535,0.491765,0.428229,0.480883
Robbery (2015),0.941810,0.878822,0.720658,0.559738,1.000000,0.882090,0.910154,0.890374,0.876923,0.921979,...,0.539265,0.480479,0.539663,0.490100,0.537360,0.510509,0.535191,0.540518,0.536243,0.552444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Unemployed_2015,0.599091,0.636717,0.520615,0.488146,0.510509,0.616817,0.527455,0.534543,0.486845,0.627330,...,0.976128,0.996663,0.976208,0.998369,0.976067,1.000000,0.976240,0.994119,0.975834,0.992322
Employed_2016,0.611273,0.625769,0.552866,0.422535,0.535191,0.620409,0.563563,0.554688,0.531243,0.634744,...,0.999677,0.969586,0.999810,0.970942,0.999927,0.976240,1.000000,0.981131,0.999940,0.983330
Unemployed_2016,0.619764,0.639510,0.536571,0.491765,0.540518,0.632624,0.555040,0.563003,0.514619,0.645438,...,0.982025,0.988718,0.982025,0.988708,0.981645,0.994119,0.981131,1.000000,0.980463,0.998682
Employed_2017,0.613073,0.627768,0.554789,0.428229,0.536243,0.622437,0.565697,0.556786,0.533367,0.636485,...,0.999423,0.968857,0.999610,0.970512,0.999773,0.975834,0.999940,0.980463,1.000000,0.982756


In [351]:
merged_dataset.columns

Index(['State', 'Raw County Data (2015)', 'Violent crime (2015)',
       'Murder and nonnegligent manslaughter (2015)', 'Rape (revised) (2015)',
       'Rape (previous def) (2015)', 'Robbery (2015)',
       'Aggravated assault (2015)', 'Property crime (2015)', 'Burglary (2015)',
       ...
       ' Unemployed_2014 ', ' Employed_2015 ', ' Unemployed_2015 ',
       ' Employed_2016 ', ' Unemployed_2016 ', 'Employed_2017',
       'Unemployed_2017', 'Geographical Area (sq mi)',
       'Housing Density (per sq mi)', 'Population'],
      dtype='object', length=109)

In [352]:
display(merged_dataset)
print("Rows/Columns:",merged_dataset.shape)

,State,Raw County Data (2015),Violent crime (2015),Murder and nonnegligent manslaughter (2015),Rape (revised) (2015),Rape (previous def) (2015),Robbery (2015),Aggravated assault (2015),Property crime (2015),Burglary (2015),...,Unemployed_2014,Employed_2015,Unemployed_2015,Employed_2016,Unemployed_2016,Employed_2017,Unemployed_2017,Geographical Area (sq mi),Housing Density (per sq mi),Population
0,Alabama,Autauga,69.0,0.0,13.0,NaN,6.0,50.0,344.0,111.0,...,1495.0,24272.0,1330.0,24593.0,1325.0,24908.0,1001.0,604.39,37.2,54571
1,Alabama,Baldwin,115.0,0.0,9.0,NaN,33.0,73.0,648.0,225.0,...,5300.0,82843.0,4862.0,85656.0,4844.0,87915.0,3652.0,2027.31,65.5,182265
2,Alabama,Bibb,7.0,0.0,2.0,NaN,1.0,4.0,41.0,20.0,...,617.0,8005.0,567.0,8050.0,557.0,8133.0,373.0,626.17,14.4,22915
3,Alabama,Blount,204.0,5.0,16.0,NaN,5.0,178.0,832.0,247.0,...,1503.0,23152.0,1321.0,23248.0,1328.0,23509.0,985.0,650.63,37.0,57322
4,Alabama,Calhoun,16.0,0.0,4.0,NaN,1.0,11.0,413.0,181.0,...,3773.0,42679.0,3231.0,42815.0,3003.0,43333.0,2249.0,612.29,88.0,118572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2769,Vermont,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,164.0,2596.0,159.0,2565.0,136.0,2555.0,125.0,675.29,7.6,6306
2770,Vermont,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,609.0,15338.0,545.0,15396.0,495.0,15445.0,489.0,692.27,21.6,28936
2771,Vermont,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1324.0,32849.0,1188.0,32804.0,1083.0,33115.0,1009.0,695.42,43.6,59534
2772,Vermont,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1093.0,28501.0,948.0,28454.0,862.0,28508.0,750.0,976.78,35.2,56670


Rows/Columns: (2774, 109)


In [355]:
merged_dataset.to_csv("/home/sbr0/Dropbox/Analysis Planning/coursera stuffs/Capstone/First-Class Restaurant Location Analysis/data sets/Counties Dataset.csv",index=False)